In [1]:
#TODO
# Include source to web search
# Include a case if the user asks for more than one action on the modelling tools in a single query (for now we'll take care of just one at a time)
# Connect all outputs of modelling tools to a decision router to verify if there was a valid selection

In [2]:
### Necessary dependencies

## Model instantiation
# %pip install transformers -U
# %pip -q install langchain-groq

## RAG node
# %pip install beautifulsoup4
# %pip install faiss-cpu

## Web search node
# %pip install -U langchain-community tavily-python

## Graph building
# %pip install -U langgraph

## Model modifications in Excel
# %pip install openpyxl

## Import api keys

In [1]:
import yaml

with open('../metadata/secrets.yml', 'r') as f:
    secrets = yaml.load(f, Loader=yaml.SafeLoader)

## Defining the model

To test it, first run 'ollama serve' in a local terminal. (necessary only for the embeddings of the RAG)

In [2]:
from langchain_groq import ChatGroq
import os

os.environ["GROQ_API_KEY"] = secrets['groq'][0]
chat_model = ChatGroq(
            model="llama3-70b-8192",
        )
json_model = ChatGroq(
            model="llama3-70b-8192",
        ).bind(response_format={"type": "json_object"})

test_model = ChatGroq(
            model="llama-3.1-70b-versatile",
        )
test_json_model = ChatGroq(
            model="llama-3.1-70b-versatile",
        ).bind(response_format={"type": "json_object"})

## Alternative model

Use this one if GROQ stops working (will need to figure out the response as JSON)

In [5]:
# from langchain_community.llms import HuggingFaceEndpoint
# from langchain_community.chat_models.huggingface import ChatHuggingFace

# llm = HuggingFaceEndpoint(repo_id="meta-llama/Meta-Llama-3-8B-Instruct", huggingfacehub_api_token=secrets['huggingface'][0])
# chat_model = ChatHuggingFace(llm=llm)

In [6]:
chat_model.invoke('Hello, who are you?')

AIMessage(content="Nice to meet you! I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation, answer questions, and even generate text. I can chat with you about a wide range of topics, from science and history to entertainment and culture. I'm constantly learning and improving, so please bear with me if I make any mistakes. What would you like to talk about?", response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 16, 'total_tokens': 119, 'completion_time': 0.327875371, 'prompt_time': 0.003940442, 'queue_time': None, 'total_time': 0.331815813}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-f8456faf-65ff-468c-8603-6092be18ea23-0')

## State

Defines the graph's state dictionary

In [3]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_query: user input
        next_query: partial query generated by the agent
        num_steps: number of steps
        selected_tool: name of the selected tool
        identified_model: name of the model identified by the agent
        rag_questions: questions used for retrieval
        tool_parameters: parameters to be used by tools
        context: list of context generated for the query
        complete_data: indicates completeness of data
        final_answer: LLM generation
    """
    initial_query : str
    query_type: str
    next_query: str
    num_steps : int
    selected_tool: str
    rag_questions : List[str]
    tool_parameters: str
    context : List[str]
    complete_data : bool
    model: str
    scenario: str
    parameter: str
    cs: str
    variable: str
    plot_type: str
    new_value: float
    selection_is_valid: bool
    history: List[dict]
    final_answer : str

In [8]:
def history_builder(history, new_message, role):
    history = history.append({"role": role, "content": new_message})
    return history

## Type identifier node

In [4]:
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [10]:
esm_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at breaking down a multi step command given by the user into many smaller
    and single step commands to be run by other LLMs.\n

    The following actions are available and can be referenced as individual actions: ['modify_model',
    'run_model', 'compare_results', 'compare_yearly_inputs', 'compare_inputs', 'plot_result',
    'plot_comparison']\n
    
    You must output a JSON with two keys 'action' and 'action_context', where action must be
    a list with the actions to be taken (in order), where each value should be part of the given array, and context 
    is the information needed for the selected action (also a list in the same order as action). If you
    don't need any extra information for an action, simply use an empty string.\n
    
    You can use the comparison actions to explain details about the model to the user.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    COMMAND : {command} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["command"],
)
esm_chain = esm_prompt | json_model | JsonOutputParser()

command = 'How would the model react if we changed the production cost of coal electricity to 800 Euro/kW?'
print(esm_chain.invoke({"command": command}))

{'action': ['modify_model', 'run_model', 'compare_results', 'plot_result'], 'action_context': ['production cost of coal electricity to 800 Euro/kW', '', '', '']}


In [11]:
type_identifier_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at identifying the type of a query provided by the user among
    the types "general", "energy_system" and "mixed".

    "general": the query is related to some generic topic, it may consist of one or more
    points that require searching for information. \n
    
    "energy_system": the query is a direct command related to the energy system model, it can
    be a request to change parameters, plot data, run simulations, or anything on this lines.
    To be characterized as this class, it should need no external information. Names of
    simulations, scenarios, parameters and any other potential name is assumed to be know by our tools. \n
    
    "mixed": the query is related to the energy system model, but it requires external data for the
    command to be complete. It MUST be related to running anything related to the energy system,
    otherwise it is not mixed. \n
    
    You must output a JSON with a single key 'query_type' containing exclusivelly the 
    selected type. \n
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY : {query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query"],
)
type_identifier_chain = type_identifier_prompt | json_model | JsonOutputParser()

# query = 'Modify the cost of CO2 in 2020 to be same price as a liter of Coca Cola'
# print(type_identifier_chain.invoke({"query": query}))

In [12]:
def type_identifier(state):
    
    print("---TYPE IDENTIFIER---")
    query = state['initial_query']
    num_steps = state['num_steps']
    num_steps += 1
    
    print(f'QUERY: {query}')

    gen = type_identifier_chain.invoke({"query": query})
    selected_type = gen['query_type']
    
    print(f'IDENTIFIED_TYPE: {selected_type}\n')
    
    return {"query_type": selected_type,
            "num_steps": num_steps}

## Energy System tool selector node

In [13]:
es_tool_selector_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading the user QUERY and routing it to the correct tool in our
    modelling system. \n

    Use the following criteria to decide how to route the query to one of our available tools: \n\n
    
    If the user asks for any modification on any particular model, select 'model_modifier'. \n
    
    If the user asks to plot anything, select 'data_plotter'. \n
    
    If the user asks to run a simulation of any particular model, select 'sim_runner'. \n

    You must output a JSON object with two keys:
    'selected_tool' containing one of the following values ['model_modificator', 'data_plotter', 'sim_runner'];
    'selected_model' containing the name of the model to be manipulated. \n
    
    If the user didn't provide a model name, fill the key 'selected_model' with 'NO_MODEL'. \n
    
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY : {query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query"],
)
es_tool_selector_chain = es_tool_selector_prompt | json_model | JsonOutputParser()

# initial_query = 'Modify the lifetime of wind power plants to 20 years'
# print(es_tool_selector_chain.invoke({"query": initial_query}))

In [14]:
def es_tool_selector(state):
    
    print("---ENERGY SYSTEM TOOL SELECTION---")
    query = state['initial_query']
    num_steps = state['num_steps']
    num_steps += 1
    
    print(f'QUERY: {query}')

    router = es_tool_selector_chain.invoke({"query": query})
    router_decision = router['selected_tool']
    identified_model = router['selected_model']
    
    print(f'SELECTED TOOL: {router_decision}')
    print(f'IDENTIFIED MODEL: {identified_model}\n')
    
    return {"selected_tool": router_decision,
            "identified_model": identified_model,
            "num_steps": num_steps}

## Model selector node

In [15]:
from os import walk

def model_selector(state):
    num_steps = state['num_steps']
    num_steps += 1
    
    print("No valid model was found for the requested action, the available models are:\n")
    
    available_models = next(walk('Models'), (None, None, []))[2]
    for i in range(len(available_models)):
        print(f'{i+1}: {available_models[i]}')
    
    selected_model = input('Please, inform the number of the desired model:\n')
    
    return {"model": available_models[int(selected_model)-1]}

In [16]:
def validated_model(state):
    return None

## Mixed node

In [17]:
mixed_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading the user QUERY and the available CONTEXT to decide if there
    is already enough information gathered to fulfill the energy system related command
    made by the user. \n
    
    You must be certain that you have all the data before deciding to send it to the
    modelling section of the pipeline. If any of the values asked by the user is not
    directly given by him, you can't consider the data complete unless you have the
    desired value in the CONTEXT. \n

    You must output a JSON object with a single key 'complete_data' containing a boolean
    on whether you have enough data for the user's request or not. \n
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY : {query} \n
    CONTEXT: {context} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query","context"],
)
mixed_chain = mixed_prompt | json_model | JsonOutputParser()

# initial_query = 'Modify the lifetime of wind power plants to be the age of Ronaldinho Gaucho plus Oprah age'
# context = ['The current age of Ronaldinho Gaucho is 44 years old', 'Oprah is 68 years old']
# print(mixed_chain.invoke({"query": initial_query, "context": context}))

In [18]:
def mixed(state):
    
    print("---TOOL SELECTION---")
    query = state['initial_query']
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1
    
    print(f'QUERY: {query}')
    print(f'CONTEXT: {context}')

    decision = mixed_chain.invoke({"query": query, "context": context})
    decision = decision['complete_data']
    
    print(f'DATA IS COMPLETE: {decision}\n')
    
    return {"complete_data": decision,
            "num_steps": num_steps}

## Tool selector node

In [19]:
tool_selector_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading a QUERY from a user and routing to our internal knowledge system\
     or directly to final answer. \n

    Use the following criteria to decide how to route the query to one of our available tools: \n\n
    
    If the user asks anything about LangSmith, you should use the 'RAG_retriever' tool.
    
    For any mathematical problem you should use 'calculator'. Be sure that you have all the necessary
    data before routing to this tool.

    If you are unsure or the person is asking a question you don't understand then choose 'web_search'

    You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web_search.
    Give a choice contained in ['RAG_retriever','calculator','web_search'].
    Return the a JSON with a single key 'router_decision' and no premable or explaination.
    Use the initial query of the user and any available context to make your decision about the tool to be used.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY : {query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query"],
)
tool_selector_chain = tool_selector_prompt | json_model | JsonOutputParser()

# initial_query = 'Please, let me know the weather in San Francisco'
# print(tool_selector_chain.invoke({"query": initial_query}))

In [20]:
def tool_selector(state):
    
    print("---TOOL SELECTION---")
    query = state['next_query']
    num_steps = state['num_steps']
    num_steps += 1
    
    print(f'QUERY: {query}')

    router = tool_selector_chain.invoke({"query": query})
    router_decision = router['router_decision']
    
    print(f'SELECTED TOOL: {router_decision}\n')
    
    return {"selected_tool": router_decision,
            "num_steps": num_steps}

## Web/RAG answer analyzer prompt

In [21]:
answer_analyzer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at summarizing a bunch of data to extract only the important bits from it.

    Given the user's QUERY and the SEARCH_RESULTS, summarize as briefly as possible the information
    searched by the user. Don't give any preamble or introduction, go directly to the summary
    of the requested information.
    
    If it helps to provide a more precise answer, you can also make use of the CONTEXT.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY: {query} \n
    SEARCH_RESULTS: {search_results} \n
    CONTEXT: {context} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["query","search_results","context"],
)
answer_analyzer_chain = answer_analyzer_prompt | chat_model | StrOutputParser()

# query = 'How much does a liter of Coca Cola cost in Brazil?'
# search = page_content="Brazil - Coca-Cola - price, May 2024. The price is 0.86 USD. The average price for all countries is 1.04 USD. The database includes 90 countries. Definition: The Coca - Cola prices are for a bottle of 0.5 l. Adjustments were made to the various measuring units across countries to arrive at a uniform measure of 0.5 l.\nBased on 90 countries included in our data base, the average price is 1.04 USD. Looking at the latest data, the lowest price was 0.22 USD (Nigeria) and the highest price was 2.60 USD (Norway). Definition: The Coca - Cola prices are for a bottle of 0.5 l.\nSee current prices by country for prices of items we do track. You can see prices only for countries where we have decent number of contributors. Prices by Country of Coke/Pepsi (0.33 liter bottle) (Restaurants)\nCoca-Cola FEMSA is the largest independent bottler of Coca-Cola products in the world, and the largest of several local bottling partners in Brazil. Within FEMSA's South America operating division (of which Brazil is the largest single market), FEMSA reported a 25.9%% increase in the average price per unit case for the first six months of 2022.\nLarge corporations. There are three main soda companies in the country. Data below is provided by Afrebras. Coca-Cola Company, which has a market share of 55%% in volume and 62%% in value. AmBev, with a market share of 19%% in volume and 21%% in value. Brasil Kirin, with a market share of 5%% in volume and 4%% in value."
# print(answer_analyzer_chain.invoke({"query": query, "search_results": search, "context": []}))

## RAG node

For now it's just a placeholder that searchs questions about LangSmith

In [22]:
## RAG QUESTIONS
search_rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best questions to ask our knowledge agent to get the best info for the customer.

    Given the INITIAL_QUERY, work out the best questions that will find the best \
    info for helping to write the final answer. Write the questions to our knowledge system not to the customer.

    Return a JSON with a single key 'questions' with no more than 3 strings of and no preamble or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_QUERY: {initial_query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_query"],
)
question_rag_chain = search_rag_prompt | json_model | JsonOutputParser()

# query = 'What are the main benefits of using LangSmith for developing a tool to levarage LLMs?'
# print(question_rag_chain.invoke({"initial_query": query}))

In [23]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough

# Load the data that will be used by the retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

# Set the embedding model
embeddings = OllamaEmbeddings(model="llama3")

# Split the data and vectorize it
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

# Define a chain to gather data and a retriever
retriever = vector.as_retriever()

In [24]:
#RAG Chain
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUESTION: {question} \n
    CONTEXT: {context} \n
    Answer:
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","context"],
)
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | rag_prompt
    | chat_model
    | StrOutputParser()
)

In [25]:
def research_info_rag(state):

    print("---RAG LANGSMITH RETRIEVER---")
    initial_query = state['next_query']
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1

    questions = question_rag_chain.invoke({"initial_query": initial_query})
    questions = questions['questions']

    rag_results = []
    for idx, question in enumerate(questions):
        print(f'QUESTION {idx}: {question}')
        temp_docs = rag_chain.invoke(question)
        print(f'ANSWER FOR QUESTION {idx}: {temp_docs}')
        question_results = question + '\n\n' + temp_docs + "\n\n\n"
        if rag_results is not None:
            rag_results.append(question_results)
        else:
            rag_results = [question_results]
    print(f'FULL ANSWERS: {rag_results}\n')
    
    processed_searches = answer_analyzer_chain.invoke({"query": initial_query, "search_results": rag_results, "context": context})
    
    return {"context": context + [processed_searches],
            "rag_questions": questions,
            "num_steps": num_steps}

## Web search node

In [26]:
## Search keywords
search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best keywords to search for in a web search to get the best info for the user.

    Given the INITIAL_QUERY, work out the best keywords that will find the info requested by the user
    The keywords should have between 3 and 5 words each, if the query allows for it.

    Return a JSON with a single key 'keywords' with no more than 3 keywords and no preamble or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_QUERY: {initial_query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_query"],
)
search_keyword_chain = search_keyword_prompt | json_model | JsonOutputParser()

# query = 'Who is the current holder of the speed skating world record on 500 meters?'
# print(search_keyword_chain.invoke({"initial_query": query}))

In [27]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os

os.environ["TAVILY_API_KEY"] = secrets['tavily'][0]
web_search_tool = TavilySearchResults()

In [28]:
def research_info_web(state):

    print("---RESEARCH INFO SEARCHING---")
    initial_query = state['next_query']
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = search_keyword_chain.invoke({"initial_query": initial_query, "context": context})
    keywords = keywords['keywords']
    full_searches = []
    for idx, keyword in enumerate(keywords):
        print(f'KEYWORD {idx}: {keyword}')
        temp_docs = web_search_tool.invoke({"query": keyword})
        if type(temp_docs) == list:
            web_results = "\n".join([d["content"] for d in temp_docs])
            web_results = Document(page_content=web_results)
        elif type(temp_docs) == dict:
            web_results = temp_docs["content"]
            web_results = Document(page_content=web_results)
        else:
            web_results = 'No results'
        print(f'RESULTS FOR KEYWORD {idx}: {web_results}')
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]
    print(f'FULL RESULTS: {full_searches}\n')
    
    processed_searches = answer_analyzer_chain.invoke({"query": initial_query, "search_results": full_searches, "context": context})
    
    print(f'PROCESSED RESULT: {processed_searches}')
    
    return {"context": context + [processed_searches],
            "num_steps": num_steps}

## Calculator node

In [29]:
## CALCULATOR
calculator_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a specialist at building JSON to do calculations using a calculator tool.
    
    You can only output a single format of JSON object consisting in two operands
    and the operation. The name of the only three keys are 'operation', 'op_1' and 'op_2' \n
    
    'operation' can only be [+,-,*,/,^]
    'op_1' and 'op_2' must be integers or float\n
    
    If you judge that the equation consists of more than one operation, solve only one,
    the calculator can be called multiple times and the other results will be solved
    later.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_QUERY: {initial_query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_query"],
)
calculator_chain = calculator_prompt | json_model | JsonOutputParser()

# query = 'How much is 27 to the power of 5 plus 7?'
# print(calculator_chain.invoke({"initial_query": query}))

In [30]:
def calculator(state):

    print("---CALCULATOR TOOL---")
    
    query = state['next_query']
    context = state['context']
    parameters = calculator_chain.invoke({"initial_query": query})
    operation = parameters['operation']
    op_1 = parameters['op_1']
    op_2 = parameters['op_2']
    num_steps = state['num_steps']
    num_steps += 1
    
    print(f'OPERATION: {operation}')
    print(f'OPERAND 1: {op_1}')
    print(f'OPERAND 2: {op_2}')

    if operation == "+":
        result = op_1 + op_2
    elif operation == "-":
        result = op_1 - op_2
    elif operation == "/":
        result = op_1 / op_2
    elif operation == "*":
        result = op_1 * op_2
    elif operation == "^":
        result = op_1 ** op_2
    else:
        result = 'ERROR'
        
    if result == 'ERROR':
        str_result = 'Unable to execute the selected operation'
    else:
        str_result = f'{op_1} {operation} {op_2} = {result}'
        
    print(f'RESULT: {str_result}\n')
        
    return {"context": context + [str_result],
            "num_steps": num_steps}

## Context analyzer node

In [31]:
## CONTEXT ANALYZER
context_analyzer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a specialist at deciding if the already available information is enough to
    fully answer the user query. \n
    
    Given a INITIAL_QUERY and the available CONTEXT, decide if the available information
    is already enough to answer the query proposed by the user. \n
    
    Your job is to coordinate the usage of many tools, one at a time. To do this you will
    decide what information you need next, with the restriction that you can only get one
    information per iteration, and request it to the pipeline. \n
    
    Your output should be a JSON object containing three keys, 'ready_to_answer',
    'next_query' and 'user_input'. 'ready_to_answer' is a boolean that indicates if all
    necessary info is present, 'next_query' is a query that you should develop so the next
    agent in the pipeline can search for the required information and 'user_input' is also
    a boolean that indicates if the question can ONLY be answered by the user. If there is
    any chance of finding the information without asking further questions to the user, leave
    this field as false.\n
    
    In the following situations you must output 'next_query' as "<KEEP_QUERY>":
    - User asks to modify parameters or characteristics of an energy system model;
    - Plotting, they don't require extra information, the tools can handle it perfectly;
    - User asks you to run a new simulation on an energy modeling system;
    - User gives you a direct command related to modelling;
    - The user asks anything about LangSmith (understand that as having the word LangSmith) \n
    
    You also have access to the last NEXT_QUERY you generated, to avoid repeating yourself.
    Never output the same 'next_query' that you've already asked in NEXT_QUERY. \n
    
    Consider that for you boolean answer the words false and true should always be written
    in full lower case. \n

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_QUERY: {initial_query} \n
    CONTEXT: {context} \n
    NEXT_QUERY: {next_query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_query","context","next_query"],
)
context_analyzer_chain = context_analyzer_prompt | json_model | JsonOutputParser()

query = 'Is my car more powerful than a GT-R R32?'
query = 'Can you suggest any mechanic near the Frankfurt area where I could make modifications in my honda civic?'
# context = ['The car owned by the user is from 2010']
print(context_analyzer_chain.invoke({"initial_query": query, "context": [], "next_query": ''}))

{'ready_to_answer': False, 'next_query': 'What is your approximate location within the Frankfurt area?', 'user_input': False}


In [32]:
def context_analyzer(state):
    print("---CONTEXT ANALYZER---")
    ## Get the state
    initial_query = state['initial_query']
    next_query = state['next_query']
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1

    output = context_analyzer_chain.invoke({"initial_query": initial_query,
                                           "next_query": next_query,
                                           "context": context
                                           })
    
    if output['next_query'] == '<KEEP_QUERY>':
        output['next_query'] = state['initial_query']
    
    if output['user_input']:
        context = context + [output['next_query']]
    
    return {"next_query": output,
            "context": context,
            "num_steps": num_steps}

## Date Getter Node

In [33]:
from datetime import datetime

def date_getter(state):

    print("---DATE GETTER TOOL---")
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1
    
    current_date = datetime.now().strftime("%d %B %Y, %H:%M:%S")
    
    result = f'The current date and time are {current_date}'
    
    print(f'CURRENT DATE: {current_date}\n')

    return {"context": context + [result],
            "num_steps": num_steps}

## Params identifier

In [34]:
params_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a specialist at identifying the correct conversion subprocess and the correct parameter
    selected by the user in his QUERY. \n
    
    As a context, you will receive two data arrays. PARAMS provides you the name of the parameters
    available to be selected. CONVERSION_SUBPROCESSES provides you the combination of 'cp' (conversion process name),
    'cin' (commodity in), 'cout' (commodity out) and 'scen' (scenario) in the format 'cp@cin@cout@scen'.\n
    
    Your goal is to output a JSON object containing three keys: 'param', 'value', 'cs_list'.
    'param' must receive the name of the selected parameter;
    'value' is the new value selected by the user;
    'cs_list' is a list with all matching conversion subprocesses (idealy only one if possible); \n
    
    NEVER MAKE UP DATA, USE ONLY DATA FROM THE GIVEN LIST. NEVER MODIFY THE SELECTED ENTRY, USE IT AS YOU
    FOUND IT IN THE LIST! \n
    
    If you can't find any match to the 'cp' name, leave the field 'cs_list' empty. If you can't find any match
    to the 'param' name, fill the field param with 'NOT_FOUND'. \n
    
    For the value required by the user, if the value is not directly stated in the QUERY, it will be
    available in the CONTEXT, use the data found there, never try to guess the desired value. If you can't find
    the value in the context leave the field 'value' empty. \n
    
    The field 'value' only accepts numeric input, unless the input given by the user contains [], in this
    case you should output it as a string. \n

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY: {query} \n
    CONTEXT: {context} \n
    PARAMS: {params} \n
    CONVERSION_SUBPROCESSES: {CSs} \n
    Answer:
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["query","context","params","CSs"],
)

params_chain = params_prompt | json_model | JsonOutputParser()

In [35]:
import pandas as pd
import numpy as np
from tabulate import tabulate

def param_selector(state):

    print("---PARAM SELECTOR---")
    query = state['initial_query']
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1

    tmap = pd.ExcelFile('Models/DEModel.xlsx')
    df = pd.read_excel(tmap,"ConversionSubProcess")

    conversion_processes = np.asarray(df.iloc[:,0].dropna())
    mask = np.where(conversion_processes != 'DEBUG')
    conversion_processes = conversion_processes[mask]
    parameters = np.asarray(df.columns[4:])

    cs = np.asarray(df.iloc[:,0:4].dropna())
    mask = np.where(cs[:,0] != 'DEBUG')
    cs = cs[mask]
    conversion_subprocesses = np.empty((len(cs),1),dtype=object)

    for i in range(len(cs)):
        conversion_subprocesses[i] = f'{cs[i,0]}@{cs[i,1]}@{cs[i,2]}@{cs[i,3]}'

    output = params_chain.invoke({"query": query, "context": context, "params": parameters, "CSs": conversion_subprocesses})
    
    print('---CONFIRM SELECTION---')
    
    cs_list = output['cs_list']
    param = output['param']
    new_value = output['value']
    data = []
    for i in range(len(cs_list)):
        elements = cs_list[i].split('@')
        data.append([i+1,elements[0],elements[1],elements[2],elements[3]])
        table = tabulate(data, headers=["Index", "CP", "CIN", "COUT", "Scen"])
    
    if len(data) == 0:
        print('No matching conversion subprocess was found.')
        cs_confirm = 'N'
    elif len(data) == 1:
        print('The following matching conversion subprocess was found:\n')
        print(table)
        cs_confirm = input('Is that correct? (Y or N)\n')
        cs_select = 0 if cs_confirm == 'Y' else 'NONE'
    else:
        print('The following conversion subprocesses were found:\n')
        print(table)
        cs_select = int(input('Input the number of the correct CS (or 0 if it\'s none of these):\n')) - 1
        cs_confirm = 'Y' if cs_select != -1 else 'N'
    
    if cs_confirm == 'N':
        print('FINAL ANSWER: No matching selection.')
        return {"num_steps": num_steps,
                "cs": 'NO_MATCH',
                "selection_is_valid": False,
                "parameter": 'NO_MATCH'}
        
    if param in parameters:
        param_confirm = input(f'You want to modify the parameter {param}, is that correct? (Y or N)\n')
    else:
        print('No matching parameter was found.')
        param_confirm = 'N'
        
    if param_confirm == 'N':
        print('FINAL ANSWER: No matching selection.')
        return {"num_steps": num_steps,
                "cs": cs_list[cs_select],
                "selection_is_valid": False,
                "parameter": 'NO_MATCH'}
    else:
        print(f'FINAL ANSWER: CS: {cs_list[cs_select]}; Param: {param}')
        return {"num_steps": num_steps,
                "cs": cs_list[cs_select],
                "new_value": new_value,
                "selection_is_valid": True,
                "parameter": param}
    
    

## Scenario identifier

In [36]:
scenario_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a specialist at identifying the correct scenario choosen by the user
    in his QUERY to have the simulation run. \n
    
    As a context, you will receive a data array called SCENARIOS, which contains
    all of the scenarios that are available to be simulated. \n
    
    Your goal is to output a JSON object containing one key called 'scenario_name' that contains
    the name of the scenario selected by the user. \n
    
    NEVER MAKE UP DATA, USE ONLY DATA FROM THE GIVEN LIST. If you can't find any match to the asked scenario,
    simply fill the key 'scenario_name' with 'NOT_FOUND'. \n

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY: {query} \n
    SCENARIOS: {scenarios} \n
    Answer:
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["query","scenarios"],
)
scenario_chain = scenario_prompt | json_model | JsonOutputParser()

query = 'Run 8760h for DEModel'
scenarios = ['Base', 'Test', 'Base4twk', 'Base8760h', 'Base8twk']
scenario_chain.invoke({"query": query, "scenarios": scenarios})

{'scenario_name': 'Base8760h'}

In [37]:
import pandas as pd
import numpy as np

def scenario_selector(state):
    print('---SCENARIO SELECTOR---')
    num_steps = state['num_steps']
    num_steps += 1
    query = state['initial_query']
    
    tmap = pd.ExcelFile('Models/DEModel.xlsx')
    df = pd.read_excel(tmap,"Scenario")
    scenarios = np.asarray(df.iloc[:,0].dropna())
    
    output = scenario_chain.invoke({'query': query, 'scenarios': scenarios})
    identified_scenario = output['scenario_name']
    print(f'IDENTIFIED SCENARIO: {identified_scenario}')
    
    if identified_scenario == 'NOT_FOUND' or not(identified_scenario in scenarios):
        print('No valid scenario was identified in the request, here are the available scenarios:\n')
        for i in range(len(scenarios)):
            print(f'{i+1} - {scenarios[i]}')
        selection = int(input('Select the desired scenario to be run (select 0 if none of these):\n'))-1
        if selection != -1:
            identified_scenario = scenarios[selection]
        else:
            identified_scenario = 'NOT_FOUND'
    
    if identified_scenario == 'NOT_FOUND':
        message = 'No valid scenario was found'
        valid = False
    else:
        message = f'Selected scenario for simulation: {identified_scenario}'
        valid = True
        
    print(message)
    return {'num_steps': num_steps,
            'scenario': identified_scenario,
            'selection_is_valid': valid,
            'final_answer': message}

## Plot identifier

In [38]:
plotter_id_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a specialist at identifying from the user's QUERY the correct plot type requested by
    the user and the desired variable the user wants to plot. \n
    
    As a context, you will receive two data arrays:
    PLOT_TYPES will provide you information about the available plot types;
    VARIABLES will provide you information about the available variables to be plotted. \n
    
    Your goal is to output a JSON OBJECT containing only two keys 'plot_type' and 'variable'.
    'plot_type' will receive the selected plot type from PLOT_TYPES, if you can't find the plot
    requested by the user in the list, fill the key with 'NOT_FOUND';
    'variable' will receive the selected variable from VARIABLES, if you can't find the
    variable requested by the user fill the key with 'NOT_FOUND'. \n
    
    NEVER MAKE UP DATA, USE ONLY DATA FROM THE GIVEN LISTS. \n

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUERY: {query} \n
    PLOT_TYPES: {plot_types} \n
    VARIABLES: {variables} \n
    Answer:
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["query","plot_types","variables"],
)
plotter_id_chain = plotter_id_prompt | json_model | JsonOutputParser()

query = 'Show me the sankey plot for DEModel with Base scenario'
plot_types = ['Bar', 'TimeSeries', 'Sankey', 'SingleValue']
variables = ['TOTEX','OPEX','CAPEX','total_annual_co2_emission','cap_active','cap_new','cap_res','pin (power input)','pout (power output)']
plotter_id_chain.invoke({"query": query, "plot_types": plot_types, "variables": variables})

{'plot_type': 'Sankey', 'variable': 'DEModel'}

In [39]:
def plot_selector(state):
    print('---PLOT SELECTOR---')
    num_steps = state['num_steps']
    num_steps += 1
    query = state['initial_query']
    
    plot_types = ['Bar', 'TimeSeries', 'Sankey', 'SingleValue']
    variables = ['TOTEX','OPEX','CAPEX','total_annual_co2_emission','cap_active','cap_new','cap_res','pin','pout']
    
    output = plotter_id_chain.invoke({"query": query, "plot_types": plot_types, "variables": variables})
    identified_plot = output['plot_type']
    identified_variable = output['variable']
    print(f'IDENTIFIED PLOT: {identified_plot}\nIDENTIFIED VARIABLE: {identified_variable}')
    
    if identified_plot == 'NOT_FOUND' or not(identified_plot in plot_types):
        print('No valid plot type was identified in the request, here are the available plot types:\n')
        for i in range(len(plot_types)):
            print(f'{i+1} - {plot_types[i]}')
        selection = int(input('Select the desired type of plot (select 0 if none of these):\n'))-1
        if selection != -1:
            identified_plot = plot_types[selection]
        else:
            identified_plot = 'NOT_FOUND'
        
    if identified_variable == 'NOT_FOUND' or not(identified_variable in variables):
        print('No valid variable was identified in the request, here are the available variables:\n')
        for i in range(len(variables)):
            print(f'{i+1} - {variables[i]}')
        selection = int(input('Select the desired variable to be plotted (select 0 if none of these):\n'))-1
        if selection != -1:
            identified_variable = plot_types[variables]
        else:
            identified_variable = 'NOT_FOUND'
    
    if identified_plot == 'NOT_FOUND' or identified_variable == 'NOT_FOUND':
        message = 'No valid plot was identified'
        valid = False
    else:
        message = f'Selected plot: {identified_plot} for {identified_variable}'
        valid = True
    
    print(message)
    
    return {'num_steps': num_steps,
            'plot_type': identified_plot,
            'variable': identified_variable,
            'selection_is_valid': valid,
            'final_answer': message}

## Model modifier node

In [40]:
from openpyxl import load_workbook

def model_modifier(state):
    print('---MODEL MODIFIER---')
    
    model = state['model']
    parameter = state['parameter']
    cs = state['cs']
    new_value = state['new_value']
    num_steps = state['num_steps']
    num_steps += 1
    
    model_file = model if '.xlsx' in model else f'{model}.xlsx'
    workbook = load_workbook(filename=f'Models/{model_file}')
    cs_sheet = workbook['ConversionSubProcess']
    
    #open workbook
    param_idx = '0'
    cs_idx = '0'
    for idx, row in enumerate(cs_sheet.rows):
        if idx == 0:
            for i in range(len(row)):
                if row[i].value == parameter:
                    param_idx = row[i].coordinate
        else:
            if f'{row[0].value}@{row[1].value}@{row[2].value}@{row[3].value}' == cs:
                cs_idx = row[0].coordinate
    if param_idx == '0' or cs_idx == '0':
        final_answer = 'Selected param or cs not found.'
        print('Selected param or cs not found.')
    else:
        print(f'Cell: {param_idx[0]}{cs_idx[1:]}')
        old_value = cs_sheet[f'{param_idx[0]}{cs_idx[1:]}'].value
        cs_sheet[f'{param_idx[0]}{cs_idx[1:]}'].value = new_value
        workbook.save(filename="Models/DEModel_modified.xlsx")
        final_answer = f'Value successfully modified from {old_value} to {new_value}'
        print(final_answer)
    return {"num_steps": num_steps,
            "final_answer": final_answer}
    

## Sim runner node

In [41]:
def sim_runner(state):
    print('---SIMULATION RUNNER---')
    
    num_steps = state['num_steps']
    num_steps += 1
    model = state['model']
    scenario = state['scenario']
    
    print(f'FINAL COMMAND: python cesm.py run {model} {scenario}\n')
    
    return {"num_steps": num_steps,
            "final_answer": 'The requested simulation was successfully submited!'}

## Plotter node

In [42]:
def plotter(state):
    print('---PLOTTER---')
    
    num_steps = state['num_steps']
    num_steps += 1
    model = state['model']
    scenario = state['scenario']
    plot_type = state['plot_type']
    variable = state['variable']
    
    print(f'FINAL COMMAND: python cesm.py plot {model} {scenario} {plot_type} {variable} \n')
    
    return {"num_steps": num_steps,
            "final_answer": 'The requested data was successfully plotted!'}

## Output generator node

In [43]:
## OUTPUT GENERATOR
output_generator_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a specialist at answering the user based on context given. \n
    
    Given the INITIAL_QUERY and a CONTEXT, generate an answer for the query
    asked by the user. You should make use of the provided information
    to answer the user in the best possible way. If you think the answer
    does not answer the user completely, ask the user for the necessary
    information if possible. \n
    
    It's important never to cite that you got it from a context, the user should
    think that you know the information.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_QUERY: {initial_query} \n
    CONTEXT: {context} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_query","context"],
)
output_generator_chain = output_generator_prompt | chat_model | StrOutputParser()

# query = 'Is my car more powerful than a GT-R R32?'
# context = 'The car owned by the user is from 2010 and has 100 hp'
# print(output_generator_chain.invoke({"initial_query": query, "context": context}))

In [44]:
def output_generator(state):
    print("---GENERATE OUTPUT---")
    ## Get the state
    initial_query = state['initial_query']
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1

    answer = output_generator_chain.invoke({"initial_query": initial_query,
                                            "context": context})
    print(f'GENERATED OUTPUT:\n{answer}\n')
    
    return {"final_answer": answer}

## Routers

In [45]:
def empty_node(state):
    return None

In [46]:
def selection_validator(state):
    selection_is_valid = state['selection_is_valid']
    selected_tool = state['selected_tool']
    
    if selection_is_valid:
        return selected_tool
    else:
        return "end_not_valid"

## Printers

In [47]:
def state_printer(state):
    """print the state"""
    print("------------------STATE PRINTER------------------")
    print(f"Num Steps: {state['num_steps']} \n")
    print(f"Initial Query: {state['initial_query']} \n" )
    print(f"Next Query: {state['next_query']} \n" )
    print(f"RAG Questions: {state['rag_questions']} \n")
    print(f"Tool Parameters: {state['tool_parameters']} \n")
    print(f"Context: {state['context']} \n" )
    return

In [48]:
import pickle
def final_answer_printer(state):
    """prints final answer"""
    print("------------------FINAL ANSWER------------------")
    print(f"Final Answer: {state['final_answer']} \n")
    history = state['history']
    history.append({"role": "assistant", "content": state['final_answer']})
    
    with open("chat_history.pkl", "wb") as f:
        pickle.dump(history, f)
    
    return state

## Conditional Edges

In [49]:
def route_to_type(state):
    """
    Route to the right path based on query type.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """
    type = state['query_type']
    
    if type == 'general':
        print("---ROUTE QUERY TO GENERAL PATH---")
        return "general"
    elif type == 'energy_system':
        print("---ROUTE QUERY TO ENERGY SYSTEM PATH---")
        return "energy_system"
    elif type == 'mixed':
        print("---ROUTE QUERY TO MIXED PATH---")
        return "mixed"

In [50]:
def route_from_mix(state):

    print("---ROUTE TO MIX---")
    data_completeness = state['complete_data']

    print(data_completeness)
    if data_completeness:
        print("---APPLY COMMAND---")
        return "complete_data"
    else:
        print("---GATHER MORE CONTEXT---")
        return "needs_data"

In [51]:
from os import walk

def validate_selected_model(state):
    identified_model = state['identified_model']
    available_models = next(walk('Models'), (None, None, []))[2]
    
    if identified_model == 'NO_MODEL' or not(f'{identified_model}.xlsx' in available_models):
        return 'select_model'
    else:
        return 'model_is_valid'

In [52]:
def route_to_es_tool(state):
    """
    Route to the necessary tool.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """
    selection = state['selected_tool']
    
    if selection == 'data_plotter':
        print("---ROUTE QUERY TO DATA PLOTTER---")
        return "data_plotter"
    elif selection == 'sim_runner':
        print("---ROUTE QUERY TO SIMULATION RUNNER---")
        return "sim_runner"
    elif selection == 'model_modifier':
        print("---ROUTE QUERY TO MODEL MODIFIER---")
        return "model_modifier"

In [53]:
def route_to_tool(state):
    """
    Route to the necessary tool.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """
    selection = state['selected_tool']
    
    if selection == 'RAG_retriever':
        print("---ROUTE QUERY TO RAG RETRIEVER---")
        return "RAG_retriever"
    elif selection == 'web_search':
        print("---ROUTE QUERY TO WEB SEARCH---")
        return "web_search"
    elif selection == 'calculator':
        print("---ROUTE QUERY TO CALCULATOR---")
        return "calculator"

In [54]:
def route_to_iterate(state):

    print("---ROUTE TO ITERATE---")
    next_query = state['next_query']

    print(next_query)
    if next_query['ready_to_answer'] or next_query['user_input']:
        print("---GENERATE FINAL ANSWER---")
        return "ready_to_answer"
    else:
        print("---GATHER MORE CONTEXT---")
        return "need_context"

## Build the graph

### Build the nodes

In [55]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("type_identifier", type_identifier)
workflow.add_node("es_tool_selector", es_tool_selector)
workflow.add_node("model_selector", model_selector)
workflow.add_node("validated_model", validated_model)
workflow.add_node("mixed", mixed)
workflow.add_node("tool_selector", tool_selector)
workflow.add_node("research_info_rag", research_info_rag) # RAG search
workflow.add_node("research_info_web", research_info_web) # web search
workflow.add_node("state_printer", state_printer)
workflow.add_node("calculator", calculator)
workflow.add_node("date_getter", date_getter)
workflow.add_node("inter_node", empty_node)
workflow.add_node("param_selector", param_selector)
workflow.add_node("scenario_selector", scenario_selector)
workflow.add_node("plot_selector", plot_selector)
workflow.add_node("model_modifier", model_modifier)
workflow.add_node("sim_runner", sim_runner)
workflow.add_node("plotter", plotter)
workflow.add_node("output_generator", output_generator)
workflow.add_node("context_analyzer", context_analyzer)
workflow.add_node("final_answer_printer", final_answer_printer)

### Add edges

In [56]:
workflow.set_entry_point("date_getter")
workflow.add_edge("date_getter", "type_identifier")
workflow.add_conditional_edges(
    "type_identifier",
    route_to_type,
    {
        "general": "context_analyzer",
        "energy_system": "es_tool_selector",
        "mixed": "mixed",
    }
)

workflow.add_conditional_edges(
    "mixed",
    route_from_mix,
    {
        "complete_data": "es_tool_selector",
        "needs_data": "context_analyzer"
    }
)

workflow.add_conditional_edges(
    "es_tool_selector",
    validate_selected_model,
    {
        "select_model": "model_selector",
        "model_is_valid": "validated_model"
    }
)

workflow.add_conditional_edges(
    "validated_model",
    route_to_es_tool,
    {
        "data_plotter": "scenario_selector",
        "sim_runner": "scenario_selector",
        "model_modifier": "param_selector"
    }
)

workflow.add_conditional_edges(
    "model_selector",
    route_to_es_tool,
    {
        "data_plotter": "scenario_selector",
        "sim_runner": "scenario_selector",
        "model_modifier": "param_selector"
    }
)

workflow.add_conditional_edges(
    "scenario_selector",
    route_to_es_tool,
    {
        "data_plotter": "plot_selector",
        "sim_runner": "inter_node"
    }
)

workflow.add_conditional_edges(
    "param_selector",
    selection_validator,
    {
        "model_modifier": "model_modifier",
        "end_not_valid": "output_generator"
    }
)

workflow.add_conditional_edges(
    "plot_selector",
    selection_validator,
    {
        "data_plotter": "plotter",
        "end_not_valid": "output_generator"
    }
)

workflow.add_conditional_edges(
    "inter_node",
    selection_validator,
    {
        "sim_runner": "sim_runner",
        "end_not_valid": "output_generator"
    }
)
workflow.add_edge("model_modifier", "output_generator")
workflow.add_edge("plotter", "output_generator")
workflow.add_edge("sim_runner", "output_generator")

workflow.add_conditional_edges(
    "context_analyzer",
    route_to_iterate,
    {
        "ready_to_answer": "output_generator",
        "need_context": "tool_selector",
    },
)

workflow.add_conditional_edges(
    "tool_selector",
    route_to_tool,
    {
        "RAG_retriever": "research_info_rag",
        "web_search": "research_info_web",
        "calculator": "calculator",
    },
)
workflow.add_edge("research_info_rag", "state_printer")
workflow.add_edge("research_info_web", "state_printer")
workflow.add_edge("calculator", "state_printer")

workflow.add_conditional_edges(
    "state_printer",
    route_to_type,
    {
        "general": "context_analyzer",
        "mixed": "mixed",
    }
)

workflow.add_edge("output_generator", "final_answer_printer")
workflow.add_edge("final_answer_printer", END)

In [57]:
# Compile
app = workflow.compile()

In [58]:
import pickle
#query = 'If I pay half the age of Tom Jobim plus the height of the Empire State for a car, how much I\'ve paid?'
#query = 'What is 10 to the power of 0.4?'
#query = 'What is the temperature and humidity in Darmstadt right now? And also, what time is it?'
#query = 'Modify the parameter X to 24 for me please'
#query = 'What are some of the most important things that happened today in past years?'
#query = 'What day is today?'
#query = 'How can LangSmith help in my project?'
#query = 'I am always coming but never arrive. What am I?'
#query = 'Change the lifetime of wind power plants to 25 years please'
#query = 'Divide the height of the Burj Khalifa by Ronaldinho Gaucho\'s age, then add the current temperature in Paris (in Celsius)'
#query = 'What are good famous and more casual board games that can be played by two players?'
#query = 'Divide the number of visitors that the Eiffel tower receives yearly by the number of cars in the city of São Paulo, Brazil'
#query = 'Change the technical lifetime of wind power plants to be the age of Olaf Scholz'
#query = 'Modify the lifetime of wind power plants to be the same value as the price of one liter of Coca Cola in Brazil.'
#query = 'Modify the investment cost power of the Biomass CHP to be the number of years michael jackson has been dead to the power of 1.5'

query = 'Is my car faster than a Ferrari?'
query = 'Of course, my car is a 1998 Honda Civic 1.6'
#query = 'Well, what would I need to make it faster than a ferrari?'
#query = 'What would be the risk to the engine when upgrading it to reach about 500hp?'
query = 'Can you suggest any mechanic near the Frankfurt area where I could ask about modifications?'
query = 'Okay, more specifically it can be in Darmstadt'
query = 'Can you tell me their address?'
#query = 'None of them exist in the map, where did you get this information from?'
#query = 'Can you use the built-in web search tool to find actual mechanics?'
# run the agent
query = 'what is the temperature in darmstadt?'
try:
    with open("chat_history.pkl", "rb") as f:
        history = pickle.load(f)
except:
    history = []
history.append({"role": "user", "content": query})
print(history)
inputs = {"initial_query": history, "next_query": '', "num_steps": 0, "context": [], "history": history}
for output in app.stream(inputs, {"recursion_limit": 50}):
    for key, value in output.items():
        print(f"Finished running <{key}> \n")

[{'role': 'user', 'content': 'what is the temperature in darmstadt?'}, {'role': 'assistant', 'content': 'As of now, the temperature in Darmstadt is 31.5°C (88.6°F), feeling like 32.3°C (90.1°F).'}, {'role': 'user', 'content': 'what is the temperature in darmstadt?'}, {'role': 'assistant', 'content': 'As of now, the temperature in Darmstadt is 31.5°C (88.6°F), feeling like 32.3°C (90.1°F).'}, {'role': 'user', 'content': 'what is the temperature in darmstadt?'}, {'role': 'assistant', 'content': 'As of now, the temperature in Darmstadt is 31.5°C (88.6°F), feeling like 32.3°C (90.1°F).'}, {'role': 'user', 'content': 'what is the temperature in darmstadt?'}, {'role': 'assistant', 'content': 'As of now, the temperature in Darmstadt is 31.5°C (88.6°F), feeling like 32.3°C (90.1°F).'}, {'role': 'user', 'content': 'what is the temperature in darmstadt?'}]
---DATE GETTER TOOL---
CURRENT DATE: 09 August 2024, 10:07:12

Finished running <date_getter> 

---TYPE IDENTIFIER---
QUERY: [{'role': 'user'

In [59]:
value['final_answer']

'As of now, the temperature in Darmstadt is 31.5°C (88.6°F), feeling like 32.3°C (90.1°F).'

In [22]:
def get_params_and_cs_list(techmap_file):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"ConversionSubProcess")

    conversion_processes = np.asarray(df.iloc[:,0].dropna())
    mask = np.where(conversion_processes != 'DEBUG')
    conversion_processes = conversion_processes[mask]
    parameters = np.asarray(df.columns[4:])
    descriptions = np.asarray(df.iloc[0,4:])
    param_n_desc = np.empty((len(parameters),1),dtype=object)
    
    for i in range(len(parameters)):
        param_n_desc[i] = f'{parameters[i]} - {descriptions[i]}'

    cs = np.asarray(df.iloc[:,0:3].dropna())
    mask = np.where(cs[:,0] != 'DEBUG')
    cs = cs[mask]
    conversion_subprocesses = np.empty((len(cs),1),dtype=object)

    for i in range(len(cs)):
        conversion_subprocesses[i] = f'{cs[i,0]}@{cs[i,1]}@{cs[i,2]}'

    return param_n_desc, conversion_subprocesses

params, CSs = get_params_and_cs_list('../CESM/Data/Techmap/DEModel.xlsx')

In [23]:
import pandas as pd
import math

def get_scenario_params(techmap_file):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"Scenario")

    base_index = df.index[df['scenario_name'] == 'Base'].tolist()[0]
    discount_rate = df['discount_rate'][base_index]
    annual_co2_limit = df['annual_co2_limit'][base_index]
    co2_price = df['co2_price'][base_index]
    
    if math.isnan(discount_rate):
        discount_rate = None
    if math.isnan(co2_price):
        co2_price = None

    return {'discount_rate': discount_rate, 'annual_co2_limit': annual_co2_limit, 'co2_price': co2_price}

def get_conversion_processes(techmap_file):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"Commodity")

    cond = df['commodity_name'] != 'Dummy'
    cond = cond & (df['commodity_name'] != 'DEBUG')
    cond = cond & (df['commodity_name'].str.contains('Help') == False)

    return df['commodity_name'][cond].tolist()

In [62]:
get_scenario_params('../CESM/Data/Techmap/DEModel.xlsx')

{'discount_rate': 0.05,
 'annual_co2_limit': '[2016 870; 2030 540; 2050 240; 2060 140]',
 'co2_price': None}

In [63]:
tmap = pd.ExcelFile('../CESM/Data/Techmap/DEModel.xlsx')
df = pd.read_excel(tmap,"Scenario")

In [64]:
if any(df.iloc[0, df.columns == 'scenario_name'] == 'Base'):
    discount_rate = df.iloc[0, df.columns == 'discount_rate'].values[0]
    co2_limit = df.iloc[0, df.columns == 'annual_co2_limit'].values[0]
    co2_price = df.iloc[0, df.columns == 'co2_price'].values[0]

In [65]:
sheet_selector = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a specialist at identifying the correct sheet to be modified based on the user's QUERY. \n
            
            You must output a JSON object with a single key 'sheet', that should receive 'conversionsubprocess'.
            You should only output 'scenario' instead if the user explicitly talks about co2 limit, co2 price
            or discount rate, these terms must be included in the query. \n
            
            The only exception to this rule is that if the QUERY leads to the selection to be 'scenario' but
            SCENARIO_READY is True, then you should output 'conversionsubprocess' anyway. \n
            
            Remember that for the sheet to be 'scenario' you need to receive explicitly the defined terms from
            the QUERY. \n

            <|eot_id|><|start_header_id|>user<|end_header_id|>
            QUERY: {query} \n
            SCENARIO_READY: {scen_ready}
            Answer:
            <|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            """,
    input_variables=["query", "scen_ready"],
)

sheet_selector_chain = sheet_selector | json_model | JsonOutputParser()

query = 'I want the energy cost of coal furnaces to be the same as biomass furnaces'
query = 'Suppose the co2 limit is killed after 2030. Would there be stranded assets?'
#query = 'What if we stopped using wind energy?'
#query = 'How will the usage of renewable energy look like if we reduce the coal usage by half?'
#query = 'what happens if we limit the CO2 to be half of todays?'
query = 'Show me what happens if we increase the discount rate by 0.1'

sheet_selector_chain.invoke({"query": query, "scen_ready": False})

{'sheet': 'scenario'}

In [66]:
scenario_param = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a specialist at modifying the scenario of a model based on the user's QUERY. \n
            
            There are three parameters you can change: discount_rate, annual_co2_limit and co2_price.
            In SCEN_PARAMS you have a dictionary with the current values for each of them, and you should
            decide how to change them to fulfill the user's request. \n
            
            You must output a JSON object with a single element 'new_values' that contains the modified
            values for the three in a three elements list. If you haven't modified some of them, simply
            output the same value you received. The order should be the same as the input.\n
            
            For the anual co2 limit, if the user request a change after a specific year you should always
            add the next year to the yearly list as a starting point for the modification, and keep the remaining
            years, just changing their values. Also, for this list the only possibility are numbers for the 
            values, not 'infinity' or anything like that. \n

            <|eot_id|><|start_header_id|>user<|end_header_id|>
            QUERY: {query} \n
            SCEN_PARAMS: {scen_params}
            Answer:
            <|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            """,
    input_variables=["query", "scen_params"],
)

scenario_param_chain = scenario_param | json_model | JsonOutputParser()

query = 'Suppose the co2 limit is killed after 2030. Would there be stranded assets?'

scen_params = {'discount_rate': 0.05, 
               'annual_co2_limit': '[2016 870; 2030 540; 2050 240; 2060 140]',
               'co2_price': ''}

scenario_param_chain.invoke({"query": query, "scen_params": scen_params})

{'new_values': [0.05, '[2016 870; 2030 540; 2031 0; 2050 0; 2060 0]', '']}

In [67]:
params_general = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a specialist at identifying the correct way to modify a model based on the user's QUERY. \n
            
            You are part of a tool where there are two other agents ready to execute their specific tasks
            related to model modification. Your goal is to identify which one to use depending on the user's
            input. \n
            
            The two possibilities are:
            1. The user provides one or more specific instructions of modifications that should be done to the model,
            with the selection of specific values for each modification;
            2. The user asks for a specific scenario where the tool should decide which parameters and subprocesses
            should be modified, as well as deciding the correct value to change the combinations to. \n
            
            Your output must be a JSON object with a single key called 'parametrization_type', where your options are
            'defined' for the first case and 'undefined' for the second. These are your only possibilities. \n

            <|eot_id|><|start_header_id|>user<|end_header_id|>
            QUERY: {query} \n
            Answer:
            <|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            """,
    input_variables=["query"],
)

params_general_chain = params_general | json_model | JsonOutputParser()

query = 'I want the energy cost of coal furnaces to be the same as biomass furnaces'

params_general_chain.invoke({"query": query})

{'parametrization_type': 'defined'}

In [68]:
params_defined = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a specialist at identifying the parameters the user wish to modify in the model, as well as the 
            conversion subprocesses and new values. \n
            
            As a context, you will receive two data arrays. PARAMS provides you the name of the parameters
            available to be selected formated as a combination of the actual parameter name and their description
            in the format 'name - description'. CONVERSION_SUBPROCESSES provides you the combination of 'cp'
            (conversion process name), 'cin' (commodity in), 'cout' (commodity out)  in the format 'cp@cin@cout'. \n
            
            The user's QUERY may contain a request to change one or more combinations of parameter and 
            conversion subprocess. \n
            
            Your goal is to output a JSON object containing a single key 'param_cs_selection', which should contain
            a list. \n
            
            The composition of the list is the following: For each combination of conversion subprocess, parameter and
            value asked by the user there should be a list with three elements in 'params_cs_selection'. Each sub list
            is composed of ['parameter', ['cs_match'], 'new_value']. The output structure you should
            always follow is [[combination], [combination], [combination], ...], and this list that comprehends
            all of the combinations of parameter, cs and value is the single element of 'params_cs_selection'.\n
            
            NEVER MAKE UP DATA, USE ONLY DATA FROM THE GIVEN LISTS. NEVER MODIFY THE SELECTED ENTRY, USE IT AS YOU
            FOUND IT IN THE LIST! THE COMBINATION 'cp@cin@cout' MUST MATCH EXACTLY WITH THE ENTRIES OF THE LIST.
            YOU CAN NEVER GET cp, cin OR cout FROM DIFFERENT ENTRIES, THEY MUST ALWAYS COME FROM THE SAME. \n
            
            For 'parameter', the value is a string and you must always output either one selected param or 'NOT_FOUND'
            if you couldn't match any of the available ones (never output more than one per combination). \n
            
            For 'cs_match' you should always output a list with the conversion suprocesses matches (in format cp@cin@cout).
            If there is a sigle matching conversion subprocess, then the list will have a single element but will still 
            be a list. And if you can't match any conversion subprocess, then you must output an empty list. Important that 
            there is a difference between asking for a generic conversion process with more than a single conversion 
            subprocess related to it (situation in which you should match all options to the combination) and asking to
            change a set of conversion subprocesses with the same conversion process name (situation in which each of
            the conversion subprocesses should be given their own combinations) \n
            
            For the 'new_value' you have four possibilities:
            1. The user specified a single value for the combination. In this case you put the value as a numeric value
            in the combination list;
            2. The user specified a value containing [] in it. In this case you should use the exact same value as a string.
            DON'T CHANGE ANYTHING IN THE VALUE. For example, the user inputs [2020 10; 2030 5; 2040 2], you should output
            exactly [2020 10; 2030 5; 2040 2] as a string for the value of this combination;
            3. There is a indirect reference to the desired value, in this case the value will probably be in the provided
            CONTEXT, check there before going to the fourth possibility;
            4. If there is no value to be found for a specific combination, simply output 'NOT_PROVIDED' for that one,
            NEVER MAKE UP VALUES. 'NOT_PROVIDED' is the only possible text that you can output as a value.\n

            <|eot_id|><|start_header_id|>user<|end_header_id|>
            QUERY: {query} \n
            CONTEXT: {context} \n
            PARAMS: {params} \n
            CONVERSION_SUBPROCESSES: {CSs} \n
            Answer:
            <|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            """,
    input_variables=["query","context","params","CSs"],
)

params_defined_chain = params_defined | json_model | JsonOutputParser()

params, CSs = get_params_and_cs_list('../models/DEModel.xlsx')
query = 'Change the maximum residual capacity for biomass power plants to [2016 10; 2030 2] and for coal furnaces to [2016 30; 2040 1]'
#query = 'Change the maximum energy output of nuclear power plants to be the same as gas power plants'
query = 'Modify the fixed OM cost of the biomass CHP and of coal CHP with electricity output to 10'
#query = 'Modify the fixed OM cost of all conversion subprocesses related to coal CHP'

params_defined_chain.invoke({"query": query, "context": ['The maximum Eout for PP_gas is 200'], "params": params, "CSs": CSs})


{'param_cs_selection': [['opex_cost_power',
   ['Biomass_CHP@Biomass@Help_Biomass_CHP',
    'Biomass_CHP@Help_Biomass_CHP@Electricity'],
   10],
  ['opex_cost_power',
   ['Coal_CHP@Coal@Help_Coal_CHP', 'Coal_CHP@Help_Coal_CHP@Electricity'],
   10]]}

In [29]:
params_undefined = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
            You are a specialist at deciding the necessary modifications in the model based on the user's USER_INPUT. \n
            
            As a context, you will receive two data arrays. PARAMS provides you the name of the parameters
            available to be selected formated as a combination of the actual parameter name and their description
            in the format 'name - description'. CONVERSION_SUBPROCESSES provides you the combination of 'cp'
            (conversion process name), 'cin' (commodity in), 'cout' (commodity out)  in the format 'cp@cin@cout'. \n
            
            Your goal is to output a JSON object containing a single key 'param_cs_selection', which should contain
            a list. This list contains each combination of conversion subprocess, parameters, values and modification
            type that you judge necessary to fulfill the user's scenario. \n
            
            For each combination of conversion subprocess, parameter, value and modification type that you define as
            necessary to change there should be a list with four elements in 'params_cs_selection'. Each of the combination
            lists is composed of [['parameters'], 'cs', ['new_values'], 'mod_type']. It MUST be in this order, you can't
            change it by any means, since it will be identified by it's order later in the pipeline. The output structure you
            should always follow is [[combination], [combination], [combination], ...], and this list that comprehends
            all of the combinations of parameter, cs, value and modification type is the single element of 'params_cs_selection'.\n
            
            NEVER MAKE UP DATA, USE ONLY DATA FROM THE GIVEN LISTS. NEVER MODIFY THE SELECTED ENTRY, USE IT AS YOU
            FOUND IT IN THE LIST! FOR THE CONVERSION SUBPROCESSES YOU MUST USE THE ENTIRE CS NAME, IN THE FORMAT (CP@CIN@COUT),
            NEVER USE ONLY A PART OF IT. \n
            
            You must analyze the user's scenario request and decide all conversion subprocesses (cp@cin@cout) that should be modified,
            as well as the parameters necessary to be modified on each of the subprocesses. Each element of 'params_cs_selection'
            should contain a list with the parameters to be modified, the selected conversion subprocess to be modified 
            and a list with all the new values. 'parameters' and 'new_values' should have the same size. \n
            
            You have only two possibilities for the new values:
            1. You want to change something to 0, in this case you output a numerical 0 in the value;
            2. In any other case you should use percentual changes through decimal numbers, these decimals will be multiplied
            with the current value of the parameter. In this case you should try to be as comprehensive as possible with the
            parameter selection, since some of the parameters may be empty, leading to problems with the multiplication. In
            other words, give as much selection options as you possibly can while trying to get the user's proposed scenario. \n
            
            For the 'mod_type' you can use two different types, 'fixed' or 'multiplier'. 'fixed' tells the tool
            that you want the value to be exactly what you specified, 'multiplier' indicates that you want the
            current value to be multiplied by the value you defined. \n
            
            Always use double quotes in the JSON object. \n

            <|eot_id|><|start_header_id|>user<|end_header_id|>
            QUERY: {query} \n
            CONTEXT: {context} \n
            PARAMS: {params} \n
            CONVERSION_SUBPROCESSES: {CSs} \n
            Answer:
            <|eot_id|>
            <|start_header_id|>assistant<|end_header_id|>
            """,
    input_variables=["query","context","params","CSs"],
)

params_undefined_chain = params_undefined | json_model | JsonOutputParser()

params, CSs = get_params_and_cs_list('../CESM/Data/Techmap/DEModel.xlsx')
query = 'What if we stopped using wind energy?'
query = 'How will the usage of renewable energy look like if we reduce the coal usage by half?'
query = 'What would change if I don’t want wind turbines?'
query = 'What if offshore wind becomes very cheap?'
query = 'What if we stopped using wind energy?'
query = 'What happens if we reduce the wind turbine power generation by half?'

params_undefined_chain.invoke({"query": query, "context": [], "params": params, "CSs": CSs})

{'param_cs_selection': [['max_eout',
   'PP_WindOff_Res@Dummy@Electricity',
   [0.5],
   'multiplier'],
  ['max_eout', 'PP_WindOff_New@Dummy@Electricity', [0.5], 'multiplier'],
  ['max_eout', 'PP_WindOn_Res@Dummy@Electricity', [0.5], 'multiplier'],
  ['max_eout', 'PP_WindOn_New@Dummy@Electricity', [0.5], 'multiplier']]}

In [70]:
debug = True
params, CSs = get_params_and_cs_list('../models/DEModel.xlsx')

scen_params = {'discount_rate': 0.05, 
               'annual_co2_limit': '[2016 870; 2030 540; 2050 240; 2060 140]',
               'co2_price': None}

query = 'Change the maximum residual capacity for biomass power plants to [2016 10; 2030 2] and for coal furnaces to [2016 30; 2040 1]'
#query = 'Change the maximum energy output of nuclear power plants to be the same as gas power plants'
query = 'Modify the fixed OM cost of the biomass CHP and of coal CHP with electricity output to 10'
#query = 'Modify the fixed OM cost of all conversion subprocesses related to coal CHP'
#query = 'What if we stopped using wind energy?'
#query = 'How will the usage of renewable energy look like if we reduce the coal usage by half?'
query = 'What would change if I don’t want wind turbines?'
#query = 'What if offshore wind becomes very cheap?'
#query = 'I want the energy cost of coal furnaces to be the same as biomass furnaces'
#query = 'Suppose the co2 limit is killed after 2030. Would there be stranded assets?'

output = sheet_selector_chain.invoke({"query": query, "scen_ready": False})

print(output)

if output['sheet'] == 'scenario':
    new_params = scenario_param_chain.invoke({"query": query, "scen_params": scen_params})
else:
    output = params_general_chain.invoke({"query": query})
    
    print(output)

    if output['parametrization_type'] == 'defined':
        new_params = params_defined_chain.invoke({"query": query, "context": ['The maximum Eout for PP_gas is 200'], "params": params, "CSs": CSs})
    else:
        new_params = params_undefined_chain.invoke({"query": query, "context": [], "params": params, "CSs": CSs})
        
print(new_params)

workbook = load_workbook(filename='../models/DEModel.xlsx')

if "new_values" in new_params:
    # The scenario sheet should be modified
    print('scenario')
    
    scen_sheet = workbook['Scenario']
    new_params = new_params['new_values']
    coords = ['','','','']
    
    for idx, row in enumerate(scen_sheet.rows):
        if idx == 0:
            for i in range(len(row)):
                if row[i].value == 'scenario_name':
                    coords[0] = row[i].coordinate[0]
                if row[i].value == 'discount_rate':
                    coords[1] = row[i].coordinate[0]
                if row[i].value == 'annual_co2_limit':
                    coords[2] = row[i].coordinate[0]
                if row[i].value == 'co2_price':
                    coords[3] = row[i].coordinate[0]
        else:
            if scen_sheet[f'{coords[0]}{idx+1}'].value == 'Base':
                for i in range(len(coords)):
                    coords[i] = f'{coords[i]}{idx+1}'
                break
    
    final_answer = []
    col_names = ['discount_rate', 'annual_co2_limit', 'co2_price']
    print(coords)
    for i in range(1,4):
        old_value = scen_sheet[coords[i]].value
        if old_value != new_params[i-1]:
            scen_sheet[coords[i]].value = new_params[i-1]
            final_answer = final_answer + [f'{col_names[i-1]} modified from {old_value} to {new_params[i-1]}']
            if debug:
                print(f'{col_names[i-1]} modified from {old_value} to {new_params[i-1]}')
            
else:
    # The conversion subprocess sheet should be modified
    print('cs')
    
    cs_sheet = workbook['ConversionSubProcess']
    
    new_params = new_params['param_cs_selection']
    params_list = []
    
    for i in range(len(new_params)):
        parameter = new_params[i][0]
        if ' - ' in parameter:
            parameter = parameter.split(' - ')[0]
        cs = new_params[i][1]
        new_value = new_params[i][2]
        
        # Defined param format: [parameter, [cs_list], new_value]
        if type(cs) == list and len(cs) > 1:
            data = []
            for j in range(len(cs)):
                elements = cs[j].split('@')
                data.append([j+1,elements[0],elements[1],elements[2]])
                table = tabulate(data, headers=["Index", "CP", "CIN", "COUT"])
            
            print('More than one match were found for one of the selected conversion subprocesses.')
            print(table)
            cs_select = int(input(f'Input the number of the correct CS (0 if none, and {len(cs)+1} if all):\n')) - 1
            
            if cs_select == len(cs):
                for j in range(len(cs)):
                    params_list = params_list + [[parameter, cs[j], new_value]]
            elif cs_select >= 0:
                params_list = params_list + [[parameter, cs[cs_select], new_value]]
        elif type(cs) == list:
            params_list = params_list + [[parameter, cs[0], new_value]]
            
        # Undefined param format: [[param_list], cs, [new_value_list]]
        if type(parameter) == list and len(parameter) > 1:
            for i in range(len(parameter)):
                params_list = params_list + [[parameter[i], cs, new_value[i]]]
        elif type(parameter) == list:
            params_list = params_list + [[parameter[0], cs, new_value[0]]]
    
    #open workbook
    final_answer = []
    for i in range(len(params_list)):
        parameter = params_list[i][0]
        if ' - ' in parameter:
            parameter = parameter.split(' - ')[0]
        cs = params_list[i][1]
        split_cs = cs.split('@')
        new_value = params_list[i][2]
        
        param_idx = '0'
        cs_idx = '0'
        
        full_cs_found = False
        cs_sub = ''
        for idx, row in enumerate(cs_sheet.rows):
            if idx == 0:
                for i in range(len(row)):
                    if row[i].value == parameter:
                        param_idx = row[i].coordinate
            else:
                if f'{row[0].value}@{row[1].value}@{row[2].value}' == cs:
                    cs_idx = row[0].coordinate
                    full_cs_found = True
                elif f'{row[0].value}@{split_cs[1]}@{row[2].value}' == cs and not full_cs_found:
                    cs_idx = row[0].coordinate
                    cs_sub = f'{row[0].value}@{split_cs[1]}@{row[2].value}'
        if param_idx == '0' or cs_idx == '0':
            final_answer = final_answer + [f'{parameter} of {cs} not found.']
            if debug:
                print(f'{parameter} for {cs} not found.')
        else:
            if not full_cs_found and len(cs_sub) > 0:
                cs = cs_sub
            old_value = cs_sheet[f'{param_idx[0]}{cs_idx[1:]}'].value
            cs_sheet[f'{param_idx[0]}{cs_idx[1:]}'].value = new_value
            
            final_answer = final_answer + [f'{parameter} of {cs} modified from {old_value} to {new_value}']
            if debug:
                print(f'{parameter} of {cs} modified from {old_value} to {new_value}')
                
try:
    workbook.save(filename="../models/DEModel_modified.xlsx")
    print(final_answer)
except:
    print('Failed to save the modifications')

{'sheet': 'conversionsubprocess'}
{'parametrization_type': 'undefined'}
{'param_cs_selection': [[['cap_max', 'cap_min'], 'PP_WindOff_Res@Dummy@Electricity', [0, 0]], [['cap_max', 'cap_min'], 'PP_WindOff_New@Dummy@Electricity', [0, 0]], [['cap_max', 'cap_min'], 'PP_WindOn_Res@Dummy@Electricity', [0, 0]], [['cap_max', 'cap_min'], 'PP_WindOn_New@Dummy@Electricity', [0, 0]]]}
cs
cap_max of PP_WindOff_Res@Dummy@Electricity modified from [2016 4.1;2030 4.1;2040 0] to 0
cap_min of PP_WindOff_Res@Dummy@Electricity modified from None to 0
cap_max of PP_WindOff_New@Dummy@Electricity modified from [2016 0;2035 355] to 0
cap_min of PP_WindOff_New@Dummy@Electricity modified from None to 0
cap_max of PP_WindOn_Res@Dummy@Electricity modified from [2016 45.4;2030 30;2040 0] to 0
cap_min of PP_WindOn_Res@Dummy@Electricity modified from None to 0
cap_max of PP_WindOn_New@Dummy@Electricity modified from [2016 0;2035 180] to 0
cap_min of PP_WindOn_New@Dummy@Electricity modified from None to 0
['cap_max of

In [71]:
def get_yearly_variations(techmap_file):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"ConversionSubProcess")

    results = []
    for col in df.columns[10:-2]:
        cond = (~df[col].isna()) & (df[col].str.contains(';'))
        values = df[col][cond].tolist()
        CPs = df['conversion_process_name'][cond].tolist()
        cin = df['commodity_in'][cond].tolist()
        cout = df['commodity_out'][cond].tolist()
        for i in range(len(values)):
            results = results + [[f'{CPs[i]}@{cin[i]}@{cout[i]}', values[i]]]
        
    return results

def get_cs_param_selection(techmap_file, cs_list, param_list):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"ConversionSubProcess")
    result = []
    
    for cs in cs_list:
        split_cs = cs.split('@')

        cond = df['conversion_process_name'] == split_cs[0]
        cond = cond & (df['commodity_in'] == split_cs[1])
        cond = cond & (df['commodity_out'] == split_cs[2])
        
        for param in param_list:
            try:
                result = result + [cs, param, df[param][cond].values[0]]
            except:
                pass

    return result

def consult_info(query, techmap_file):
    consult_type = query['consult_type']
    
    if consult_type == 'yearly_variation':
        info = get_yearly_variations(techmap_file)
    elif consult_type == 'cs_param_selection':
        info = get_cs_param_selection(techmap_file, query['cs'], query['param'])
    else:
        info = 'Consult type not recognized'
    
    return info

In [72]:
consult_model_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a specialist at identifying from the user's QUERY the correct consultation that
        should be made to the model to return the required information to the user. \n
        
        You'll receive CONVERSION_PROCESSES, CONVERSION_SUBPROCESSES, PARAMETERS and SCENARIO_INFO
        as context, as well as MODEL_INFOS, which is a list of information about the model that was
        already gathered. You should always focus on completing the necessary information, while
        avoiding to get information that already exists. \n
        
        CONVERSION_PROCESSES is a list with the name of all conversion processes. They are the general 
        processes of energy conversion and generation, better fragmentalized in CONVERSION_SUBPROCESSES. \n
        
        CONVERSION_SUBPROCESSES is presented to you in the format "'cp'@'cin'@'cout' - 'description'" and
        describes the processes of energy exchange, such as gas to heat, or gas to electricity, for each
        of the conversion processes. \n
        
        PARAMETERS is a list with all parameters that can be modeled for the conversion subprocesses along
        with their descriptions. \n
        
        SCENARIO_INFO is a dictionary with the relevant information about the scenario to consider
        if asked by the user. It contains the discount rate of the model, the anual limits of CO2 emission
        and the defined price for CO2. \n
        
        Beyond the already available information, you can ask for two other types of information,
        ['yearly_variation', 'cs_param_selection']. \n
        
        - yearly_variation: provides you with all parameters of specific conversion subprocesses that
        have yearly variation. The parameter is displayed as 'conversion_process'@'cin'@'cout'@'parameter', and
        the value comes as a list with years and values, such as [2016 10; 2020 20; 2030 30] for example. This
        would show you yearly variation of parameters for different conversion subprocesses;
        - cs_param_selection: allows you to choose a set of conversion subprocesses and parameters
        to get their values from the model. \n
        
        Your only output should be a JSON with three keys, 'consult_type', 'cs' and 'param'. 'consult_type' must
        be 'yearly_variation' or 'cs_param_selection, 'cs' and 'param' are optional, and should only be used
        in the 'cs_param_selection' case. \n
        
        Whenever you need to use 'cs_param_selection', you should get 'cs' from CONVERSION_SUBPROCESS and
        'param' from PARAMETERS. You MUST NOT modify the entries, you should use them exactly as they are given
        to you in the input lists, you are also NOT ALLOWED to guess CSs or params, use only the ones
        available in the lists. The values in 'cs' and 'param' should always be lists, even if you want
        to know a single value. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        QUERY: {query} \n
        CONVERSION_PROCESSES: {cp} \n
        CONVERSION_SUBPROCESSES: {cs} \n
        PARAMETERS: {param} \n
        SCENARIO_INFO: {scen_infos} \n
        MODEL_INFOS: {model_infos} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["query","cp","cs","param","scen_infos","model_infos"],
)

sum_up_info_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an specialist at sumarizing model information based on the user's QUERY. \n
        
        You'll receive CONVERSION_PROCESSES, CONVERSION_SUBPROCESSES, PARAMETERS SCENARIO_INFO
        and CONSULTED_INFO as context. \n
        
        CONVERSION_PROCESSES is a list with the name of all conversion processes. They are the general 
        processes of energy conversion and generation, better fragmentalized in CONVERSION_SUBPROCESSES. \n
        
        CONVERSION_SUBPROCESSES is presented to you in the format "'cp'@'cin'@'cout' - 'description'" and
        describes the processes of energy exchange, such as gas to heat, or gas to electricity, for each
        of the conversion processes. \n
        
        PARAMETERS is a list with all parameters that can be modeled for the conversion subprocesses along
        with their descriptions. \n
        
        SCENARIO_INFO is a dictionary with the relevant information about the scenario to consider
        if asked by the user. It contains the discount rate of the model, the anual limits of CO2 emission
        and the defined price for CO2. \n
        
        CONSULTED_INFO contains possible extra information about the model that you may need to evaluate
        the answer. \n
        
        Your goal with all this context is to sumarize the information requested by the user about the
        model based on the QUERY. Make the summarization the most natural possible, without citing the name
        of the source inputs. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        QUERY: {query} \n
        CONVERSION_PROCESSES: {cp} \n
        CONVERSION_SUBPROCESSES: {cs} \n
        PARAMETERS: {param} \n
        SCENARIO_INFO: {scen_infos} \n
        CONSULTED_INFO: {info} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["query","cp","cs","param","scen_infos","info"],
)

consult_model_chain = consult_model_prompt | json_model | JsonOutputParser()
sum_up_info_chain = sum_up_info_prompt | chat_model | StrOutputParser()

model = '../models/DEModel.xlsx'
params, CSs = get_params_and_cs_list(model)
CPs = get_conversion_processes(model)
scen_infos = get_scenario_params(model)
model_info = []

query = 'What energy forms were modeled? And what is the CO2 limit defined for 2040?'
query = 'what are the dominant changes in 2050 compared to now? What is the main message?'

output = consult_model_chain.invoke({"query": query,"cp": CPs,"cs": CSs,"param": params,"scen_infos": scen_infos,"model_infos": model_info})

print(output)

info = consult_info(output, '../models/DEModel.xlsx')

print(info)

sum_up_info_chain.invoke({"query": query, "cp": CPs,"cs": CSs,"param": params,"scen_infos": scen_infos, "info": info})



{'consult_type': 'cs_param_selection', 'cs': ['PP_Biomass@Biomass@Electricity', 'PP_Coal@Coal@Electricity', 'PP_Gas@Gas@Electricity'], 'param': ['efficiency', 'technical_availability', 'cap_max']}
['PP_Biomass@Biomass@Electricity', 'efficiency', 0.3, 'PP_Biomass@Biomass@Electricity', 'technical_availability', 0.86, 'PP_Biomass@Biomass@Electricity', 'cap_max', '[2016 9.9138;2017 NaN]', 'PP_Coal@Coal@Electricity', 'efficiency', 0.38, 'PP_Coal@Coal@Electricity', 'technical_availability', 0.86, 'PP_Coal@Coal@Electricity', 'cap_max', '[2016 20.4;2017 NaN]', 'PP_Gas@Gas@Electricity', 'efficiency', 0.4, 'PP_Gas@Gas@Electricity', 'technical_availability', 0.95, 'PP_Gas@Gas@Electricity', 'cap_max', '[2016 12.5;2017 NaN]']


'Based on the provided context, the dominant changes in 2050 compared to now are primarily driven by the transition to a more sustainable energy system. The main message is that there is a significant shift towards renewable energy sources and a decrease in fossil fuel usage.\n\nIn 2050, the energy system is expected to undergo significant changes, with a focus on reducing greenhouse gas emissions and meeting the annual CO2 limit of 240. This is achieved through the increased adoption of renewable energy sources, such as biomass, solar, and wind power, which replace traditional fossil fuels like coal, lignite, and natural gas.\n\nThe power sector is expected to undergo a significant transformation, with a shift towards more decentralized and efficient power generation. The use of combined heat and power (CHP) systems, which can generate both electricity and heat, is expected to increase.\n\nIn the transportation sector, there is a shift towards more sustainable propulsion methods, such

## RAG node

Updating to use PDF

In [4]:
## RAG QUESTIONS
search_rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best questions to ask our knowledge agent to get the best info for the customer.

    Given the INITIAL_QUERY, work out the best questions that will find the best \
    info for helping to write the final answer. Write the questions to our knowledge system not to the customer.

    Return a JSON with a single key 'questions' with no more than 3 strings of and no preamble or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_QUERY: {initial_query} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_query"],
)
question_rag_chain = search_rag_prompt | json_model | JsonOutputParser()

In [44]:
# query = 'What are the main benefits of using LangSmith for developing a tool to levarage LLMs?'
# print(question_rag_chain.invoke({"initial_query": query}))
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough

# Load the data that will be used by the retriever
loader = PyPDFLoader('../rag_source/energies-14-08084-v2.pdf')
pages = loader.load()

# Set the embedding model
embeddings = OllamaEmbeddings(model="llama3")

# Split the data and vectorize it
# text_splitter = RecursiveCharacterTextSplitter()
# documents = text_splitter.split_documents(pages)
# vector = FAISS.from_documents(documents, embeddings)

# # Define a chain to gather data and a retriever
# retriever = vector.as_retriever()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)
vectorstore = FAISS.from_documents(chunks, embeddings)

#retriever = vectorstore.as_retriever()

In [45]:
def retrieve(query, embeddings, doc_embeddings):
    vectorquery = embeddings.embed_query(query)
    docs = doc_embeddings.similarity_search_with_score(query,k=5)
    
    return docs

In [46]:
# Define a chain to gather data and a retriever
#docs = retriever.invoke("How can you describe combined heat and power?")

query = "How do we determine today's electricity consumption?"
docs = retrieve(query, embeddings, vectorstore)

In [52]:
print(docs[4][0].page_content)

intensity resulting from simulated future electricity mixes forms a key factor for determin-
ing suitable emission reduction measures in local neighborhoods that are interlinked with
their environment.


In [117]:
pages[19]

Document(page_content='Energies 2021 ,14, 8084 17 of 18\nReferences\n1. Bründlinger, T.; König, J.E.; Frank, O.; Gründig, D.; Jugel, C.; Kraft, P .; Krieger, O.; Mischinger, S.; Prein, P .; Seidl, H.; et al. Dena-\nLeitstudieIntegrierte Energiewende—Impulse für die Gestaltung des Energiesystems bis 2050. 2018. Available online: https://\nwww.dena.de/newsroom/publikationsdetailansicht/pub/dena-leitstudie-integrierte-energiewende-ergebnisbericht/ (accessed\non 29 November 2021)\n2. Gerbert, P .; Herhold, P .; Burchardt, J.; Schönberger, S.; Rechenmacher, F.; Kirchner, A.; Kemmler, A.; Wünsch, M. Klimapfade für\nDeutschland ; BCG, The Boston Consulting Group: München, Germany, 2018.\n3. Ausfelder, F.; Drake, F.D.; Erlach, B.; Fischedick, M.; Henning, H.M.; Kost, C.; Münch, W. Sektorkopplung—Untersuchungen und\nUberlegungen zur Entwicklung eines Integrierten Energiesystems (Schriftenreihe Energiesysteme der Zukunft) ; DECHEMA eV: München,\nGermany, 2017.\n4. Morrison, R. Energy system mode

In [ ]:
#RAG Chain
rag_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    QUESTION: {question} \n
    CONTEXT: {context} \n
    Answer:
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question","context"],
)
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | rag_prompt
    | chat_model
    | StrOutputParser()
)

In [ ]:
def research_info_rag(state):

    print("---RAG LANGSMITH RETRIEVER---")
    initial_query = state['next_query']
    context = state['context']
    num_steps = state['num_steps']
    num_steps += 1

    questions = question_rag_chain.invoke({"initial_query": initial_query})
    questions = questions['questions']

    rag_results = []
    for idx, question in enumerate(questions):
        print(f'QUESTION {idx}: {question}')
        temp_docs = rag_chain.invoke(question)
        print(f'ANSWER FOR QUESTION {idx}: {temp_docs}')
        question_results = question + '\n\n' + temp_docs + "\n\n\n"
        if rag_results is not None:
            rag_results.append(question_results)
        else:
            rag_results = [question_results]
    print(f'FULL ANSWERS: {rag_results}\n')
    
    processed_searches = answer_analyzer_chain.invoke({"query": initial_query, "search_results": rag_results, "context": context})
    
    return {"context": context + [processed_searches],
            "rag_questions": questions,
            "num_steps": num_steps}

## Trying Llama-parse

In [55]:
os.environ["LLAMA_CLOUD_API_KEY"] = secrets['llama-cloud'][0]

In [3]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

In [1]:
##### LLAMAPARSE #####
from llama_parse import LlamaParse

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import OllamaEmbeddings
#
from groq import Groq
from langchain_groq import ChatGroq
#
import joblib
import os
import nest_asyncio  # noqa: E402
nest_asyncio.apply()

In [4]:
def load_or_parse_data():
    data_file = "../rag_source/parsed_data.pkl"

    if os.path.exists(data_file):
        # Load the parsed data from the file
        parsed_data = joblib.load(data_file)
    else:
        # Perform the parsing step and store the result in llama_parse_documents
        parsingInstructionUber10k = """The provided document is a scientific paper related to energy systems
        and energy transition. There are many tables and formulas that can be used as data sources.
        Try to be precise while answering the questions"""
        parser = LlamaParse(result_type="markdown",
                            parsing_instruction=parsingInstructionUber10k,
                            max_timeout=5000,)
        llama_parse_documents = parser.load_data("../rag_source/energies-14-08084-v2.pdf")


        # Save the parsed data to a file
        print("Saving the parse results in .pkl format ..........")
        joblib.dump(llama_parse_documents, data_file)

        # Set the parsed data to the variable
        parsed_data = llama_parse_documents

    return parsed_data

In [14]:
# Create vector database
def create_vector_database():
    """
    Creates a vector database using document loaders and embeddings.

    This function loads urls,
    splits the loaded documents into chunks, transforms them into embeddings using OllamaEmbeddings,
    and finally persists the embeddings into a Chroma vector database.

    """
    # Call the function to either load or parse the data
    llama_parse_documents = load_or_parse_data()
    
    try:
        os.remove('../rag_source/output.md')
    except:
        pass

    with open('../rag_source/output.md', 'a') as f:  # Open the file in append mode ('a')
        for doc in llama_parse_documents:
            f.write(doc.text + '\n')

    markdown_path = "../rag_source/output.md"
    loader = UnstructuredMarkdownLoader(markdown_path)

    #loader = DirectoryLoader('data/', glob="**/*.md", show_progress=True)
    documents = loader.load()
    # Split loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    #len(docs)
    print(f"length of documents loaded: {len(documents)}")
    print(f"total number of document chunks generated :{len(docs)}")
    #docs[0]

    # Initialize Embeddings
    #embed_model = OllamaEmbeddings(model="llama3")
    embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

    # Create and persist a Chroma vector database from the chunked documents
    vs = Chroma.from_documents(
        documents=docs,
        embedding=embed_model,
        persist_directory="chroma_db_llamaparse2",  # Local mode with in-memory storage only
        collection_name="rag"
    )

    #query it
    #query = "what is the agend of Financial Statements for 2022 ?"
    #found_doc = qdrant.similarity_search(query, k=3)
    #print(found_doc[0][:100])
    #print(qdrant.get())

    print('Vector DB created successfully !')
    return vs,embed_model

In [15]:
vs,embed_model = create_vector_database()

chat_model = ChatGroq(temperature=0,
                      model_name="llama3-70b-8192",)

length of documents loaded: 1
total number of document chunks generated :61


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Vector DB created successfully !


In [18]:
vectorstore = Chroma(embedding_function=embed_model,
                    persist_directory="chroma_db_llamaparse2",
                    collection_name="rag")
#
retriever=vectorstore.as_retriever(search_kwargs={'k': 3})

In [20]:
docs = retriever.invoke('How to determine todays energy consumption?')

In [27]:
print(docs[2].page_content)

Fractions of Input Energy form Consumption, κpin,y/κpin,y

These fractions are used to model CHPs. We consider an electricity-driven operation with electrical efficiency ηel and maximal overall efficiency ηCHP. Each CHP is defined by four different processes, joined by an auxiliary commodity cp aux. More specifically, a process transforms the fuel into cp aux, and the other three use cp aux to supply: (i) the exogenous energy form to represent the conversion losses, with κpin,y equal to 1 − ηCHP; (ii) Electricity, with κpin,y/κpin,y equal to ηel; and (iii) LTIH, here no limitation of the energy consumption fraction is necessary. The efficiencies are taken from [38] for biomass and from [35] for coal and gas CHPs.

2.3.2. Model-Instance-Specific Parameters

Energy Demands and Other Annual Energy Output Limitations, ep,y out/ep,yout


In [18]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt
#
prompt = set_custom_prompt()
prompt

########################### RESPONSE ###########################
PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n")

PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n")

In [20]:
qa = RetrievalQA.from_chain_type(llm=chat_model,
                               chain_type="stuff",
                               retriever=retriever,
                               return_source_documents=True,
                               chain_type_kwargs={"prompt": prompt})

In [22]:
qa.invoke({"query": "How to determine todays energy consumption?"})

{'query': 'How to determine todays energy consumption?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='considering the different heating technologies for buildings. The model includes gas, oil, biomass, and resistive heaters as well as heat pumps and district heating. LTIH is the input commodity for District Heating.', metadata={'source': '../rag_source/output.md'}),
  Document(page_content='considering the different heating technologies for buildings. The model includes gas, oil, biomass, and resistive heaters as well as heat pumps and district heating. LTIH is the input commodity for District Heating.', metadata={'source': '../rag_source/output.md'}),
  Document(page_content='considering the different heating technologies for buildings. The model includes gas, oil, biomass, and resistive heaters as well as heat pumps and district heating. LTIH is the input commodity for District Heating.', metadata={'source': '../rag_source/output.md'})]}

## Testing with Qdrant

In [6]:
import os
os.environ["LLAMA_CLOUD_API_KEY"] = secrets['llama-cloud'][0]

In [38]:
from llama_parse import LlamaParse
import glob
import nest_asyncio; nest_asyncio.apply()


parse_documents = LlamaParse(result_type='markdown').load_data(glob.glob('../rag_source/*.pdf'))

Parsing files: 100%|██████████| 1/1 [00:03<00:00,  3.03s/it]


In [7]:
import qdrant_client

client = qdrant_client.QdrantClient(api_key=secrets['qdrant']['api-key'], url=secrets['qdrant']['url'])

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

modelPath = "sentence-transformers/all-MiniLM-l6-v2"
embed_model = HuggingFaceEmbedding(model_name=modelPath)

In [9]:
from langchain_groq import ChatGroq
import os

os.environ["GROQ_API_KEY"] = secrets['groq'][0]
llm = ChatGroq(model="llama3-70b-8192")

In [10]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.llm = llm

In [54]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

vector_store = QdrantVectorStore(client=client, collection_name='llama_parse_qdrant_rag')
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents=parse_documents, storage_context=storage_context, show_progress=True)

Parsing nodes:   0%|          | 0/18 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/19 [00:00<?, ?it/s]

In [11]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

vector_store = QdrantVectorStore(client=client, collection_name='llama_parse_qdrant_rag')
test = VectorStoreIndex.from_vector_store(vector_store, embed_model)

In [17]:
query_engine = test.as_query_engine()

query = "How can we define combined heat and power processes?"
response = query_engine.query(query)

/home/gutriv/miniconda3/envs/llama/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [18]:
response.response

'Combined heat and power (CHP) processes can be defined as multi-energy conversion processes that simultaneously supply electricity and heat. The interdependence of the output commodities and the related efficiencies are considered, where the overall efficiency of a CHP process describes the ratio between the output of both energy forms and the fuel consumption.'

: 

In [30]:
import pickle
import glob

pdf_list = [f.split('/')[-1] for f in glob.glob('../rag_source/*.pdf')]

try:
    with open('../rag_source/metadata.pickle', 'rb') as handle:
        old_pdf_list = pickle.load(handle)
    update_collection = False if old_pdf_list == pdf_list else True
    print('1')
except:
    update_collection = True

if update_collection:
    print('2')
    with open('../rag_source/metadata.pickle', 'wb') as handle:
        pickle.dump(pdf_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

1


## Plotter

In [12]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from CESM.core.plotter import Plotter, PlotType
from CESM.core.data_access import DAO
import sqlite3

In [28]:
runs_dir_path = '../CESM/Runs'
simulation = 'DEModel-Base'
fname_model = 'db.sqlite'
db_path = os.path.join(runs_dir_path, simulation, fname_model)
conn = sqlite3.connect(db_path)
dao = DAO(conn)
plotter = Plotter(dao)
plot_types = [i for i in PlotType.__dict__.keys() if not i.startswith('__')]
available_plots = {}
for plot_type in plot_types:
    p_type = getattr(PlotType, plot_type)
    available_plots[plot_type] = list(p_type.__members__.keys())
    
commodities = [str(c) for c in dao.get_set("commodity")]
      # remove Dummy
commodities.remove('Dummy')
years = [int(y) for y in dao.get_set("year")]

In [35]:
available_plots

{'Bar': ['ENERGY_CONSUMPTION',
  'ENERGY_PRODUCTION',
  'ACTIVE_CAPACITY',
  'NEW_CAPACITY',
  'CO2_EMISSION',
  'PRIMARY_ENERGY'],
 'TimeSeries': ['ENERGY_CONSUMPTION',
  'ENERGY_PRODUCTION',
  'POWER_CONSUMPTION',
  'POWER_PRODUCTION'],
 'Sankey': ['SANKEY'],
 'SingleValue': ['CAPEX', 'OPEX', 'TOTEX']}

In [10]:
f = plotter.plot_sankey(years[1])
f.show()

### Plot node

In [39]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from CESM.core.plotter import Plotter, PlotType
from CESM.core.data_access import DAO
import sqlite3

runs_dir_path = '../CESM/Runs'
simulation = 'DEModel-Base'
fname_model = 'db.sqlite'

plot_results_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an specialist at deciding the correct type of plot to show to the user
        based on simulation results that we have access. \n
        
        Based on the USER_INPUT you should either identify the plot requested by the user
        directly, or try to guess the correct ones if the USER_INPUT is a bit more generic
        and don't specify any plot. \n
        
        To help you, you have access to a list called AVAILABLE_PLOTS where you have all
        plotting possibilities. Each of the sublists is a type of plot that you can request,
        some of them have elements show as REQUIRED, that means that they need extra information
        to the plot to work. The format of the sublists is ['plot_type', 'plot_subtype', 'commodity', 'year']
        and you can find the possibilities for commodity and year in COMMODITIES AND YEARS. 
        They MUST be filled in the plots that have them as REQUIRED. If it's not required you
        can keep the values empty. \n
        
        You must output a JSON object containing a single key 'plots'. The value of this key
        will be a list of lists, where each sublist consists of a plot selection. The format of
        the sublists is ['plot_type', 'plot_subtype', 'commodity', 'year'], it must stay in this
        exact order. \n
        
        All commodities and years that you select MUST be in COMMODITIES and YEARS, you are not
        allowed to guess any of them, since the tool cannot plot data for inexistent commodities
        or years. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT: {user_input} \n
        AVAILABLE_PLOTS: {available_plots} \n
        COMMODITIES: {commodities} \n
        YEARS: {years} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["user_input","available_plots","commodities","years"],
)

plot_results_chain = plot_results_prompt | json_model | JsonOutputParser()

db_path = os.path.join(runs_dir_path, simulation, fname_model)
conn = sqlite3.connect(db_path)
dao = DAO(conn)
plotter = Plotter(dao)

available_plots = [['Bar', 'ENERGY_CONSUMPTION', 'REQUIRED', ''],
  ['Bar', 'ENERGY_PRODUCTION', 'REQUIRED', ''],
  ['Bar', 'ACTIVE_CAPACITY', 'REQUIRED', ''],
  ['Bar', 'NEW_CAPACITY', 'REQUIRED', ''],
  ['Bar', 'CO2_EMISSION', '', ''],
  ['Bar', 'PRIMARY_ENERGY', '', ''],
  ['TimeSeries', 'ENERGY_CONSUMPTION', 'REQUIRED', 'REQUIRED'],
  ['TimeSeries','ENERGY_PRODUCTION', 'REQUIRED', 'REQUIRED'],
  ['TimeSeries','POWER_CONSUMPTION', 'REQUIRED', 'REQUIRED'],
  ['TimeSeries','POWER_PRODUCTION', 'REQUIRED', 'REQUIRED'],
  ['Sankey', 'SANKEY', '', 'REQUIRED'],
  ['SingleValue', 'CAPEX', '', ''],
  ['SingleValue', 'OPEX', '', ''],
  ['SingleValue', 'TOTEX', '', '']]
    
commodities = [str(c) for c in dao.get_set("commodity")]
commodities.remove('Dummy')

years = [int(y) for y in dao.get_set("year")]

query = 'Show me all the sankey plot of the model until 2040'
query = 'What happens to the model if we remove all the renewable sources in 2030?'
query = 'Show me the active capacity of coal and biomass'

output = plot_results_chain.invoke({"user_input": query,
                                     "available_plots": available_plots,
                                     "commodities": commodities,
                                     "years": years})

print(output)

selected_plots = output['plots']

for plot in selected_plots:
    plot_type = plot[0]
    plot_subtype = plot[1]
    commodity = plot[2]
    year = plot[3]
    
    p_type = getattr(PlotType, plot_type)
    
    if plot_type == 'Bar':
        # Combination
        if plot_subtype in  ['PRIMARY_ENERGY', 'CO2_EMISSION']:
            plotter.plot_bars(getattr(p_type, plot_subtype))
        else:
            plotter.plot_bars(getattr(p_type, plot_subtype), commodity=commodity)

    elif plot_type == 'TimeSeries':
        plotter.plot_timeseries(getattr(p_type, plot_subtype), year=year, commodity=commodity)
    
    elif plot_type == 'Sankey':
        f = plotter.plot_sankey(year)
        f.show()

    elif plot_type == 'SingleValue':
        plotter.plot_single_value([getattr(p_type, plot_subtype)])

{'plots': [['Bar', 'ACTIVE_CAPACITY', 'Coal', ''], ['Bar', 'ACTIVE_CAPACITY', 'Biomass', '']]}


## Compare results

In [145]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from CESM.core.data_access import DAO
import sqlite3

runs_dir_path = '../CESM/Runs'
simulation_base = 'DEModel-Base'
simulation_new = 'DEModelMock-Base'
fname_model = 'db.sqlite'

db_path_base = os.path.join(runs_dir_path, simulation_base, fname_model)
db_path_new = os.path.join(runs_dir_path, simulation_new, fname_model)
conn_base = sqlite3.connect(db_path_base)
conn_new = sqlite3.connect(db_path_new)
dao_base = DAO(conn_base)
dao_new = DAO(conn_new)

non_t_variables = ["Cap_new",
                   "Cap_active",
                   "Cap_res",
                   "Eouttot",
                   "Eintot",
                   "E_storage_level_max"]

single_values = ["OPEX",
                 "CAPEX",
                 "TOTEX"]

for idx, variable in enumerate(non_t_variables):
    df_value_base = dao_base.get_as_dataframe(variable)
    df_value_new = dao_new.get_as_dataframe(variable)
    if idx == 0:
        df_base = df_value_base.rename(columns={'value': variable})
        df_new = df_value_new.rename(columns={'value': variable})
    else:
        df_base[variable] = df_value_base['value']
        df_new[variable] = df_value_new['value']

for idx, variable in enumerate(single_values):
    df_value_base = dao_base.get_as_dataframe(variable)
    df_value_new = dao_new.get_as_dataframe(variable)
    if idx == 0:
        df_single_base = df_value_base.rename(columns={'value': variable})
        df_single_new = df_value_new.rename(columns={'value': variable})
    else:
        df_single_base[variable] = df_value_base['value']
        df_single_new[variable] = df_value_new['value']

In [146]:
variations_single = (df_single_new / df_single_base - 1) * 100

In [147]:
import numpy as np
variations = ((df_new.iloc[:,4:] / df_base.iloc[:,4:] - 1) * 100).replace([np.inf, np.nan], 0).apply(np.int64)
df_variations = df_new.copy()
df_variations[non_t_variables] = variations[non_t_variables]
df_variations['cs'] = df_variations[['cp','cin','cout']].agg('@'.join, axis=1)

In [149]:
years = np.unique(df_variations['Year'])

In [168]:
variations_dict = {var: {year: [] for year in years} for var in non_t_variables}

for variable in non_t_variables:
    for year in years:
        for index, row in df_variations.loc[(df_variations['Year']==year) & (df_variations[variable]!=0)].iterrows():
            if row['cs'] == 'DEBUG@Dummy@DEBUG':
                continue
            entry = f'{row["cs"]} = {min(row[variable], 500)}'
            variations_dict[variable][year].append(entry)

In [169]:
compare_results_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an specialist at analyzing the variations in the model's output variables to summarize
        the most relevant information given the USER_INPUT and the OUTPUT_VARIATIONS. \n
        
        OUTPUT_VARIATIONS is a dictionary that contains the percentual variation of the output variables
        of a model after the user modified it and ran it again. The first layer is has the output variables
        as the key, the second has each year of the simulation as a key, and finally, the values of these
        are the the variations for each subprocess (represented by a combination of 
        'conversion_process'@'commodity_in'@'commodity_out'). \n
        
        You must consider that the model was modified to account for the user's modification request shown
        in USER_INPUT, then the modified model was simulated and the variation you have available in
        OUTPUT_VARIATIONS is the variation between the original model and the model with the modifications
        requested by the user. Using this context you should be able to give the user the main insights
        about the scenario that he requested. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT: {user_input} \n
        OUTPUT_VARIATIONS: {output_variations} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["user_input","output_variations"],
)

compare_results_chain = compare_results_prompt | test_model | StrOutputParser()

user_input = "What would happen if we progressivelly reduced the CO2 limits of the model?"

output = compare_results_chain.invoke({"user_input": user_input, "output_variations": variations_dict})

In [6]:
import pandas as pd
import numpy as np

def get_params_and_cs_list(techmap_file):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"ConversionSubProcess")
    
    # Put the column names (parameters) and the first row (descriptions) together in an array
    parameters = df.columns[10:-2].to_list()
    descriptions = df.iloc[0,10:-2].to_list()
    param_n_desc = []
    for i in range(len(parameters)):
        param_n_desc.append(f'{parameters[i]} - {descriptions[i]}')
        
    # Filter the conversion processes and generate a new column with the full name 'cp@cin@cout'
    df = df.loc[(df['conversion_process_name'] != 'DEBUG') & (df['conversion_process_name'].notnull())]
    df = df.iloc[:,0:3]
    df['cs'] = df[['conversion_process_name', 'commodity_in', 'commodity_out']].agg('@'.join, axis=1)
    
    return param_n_desc, df['cs'].to_list()

def get_populated_params_and_cs_list(techmap_file, cs_selection):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"ConversionSubProcess")

    new_row = [col if type(df.iloc[0][col]) != str else f"{col} - {df.iloc[0][col]}" for col in df.columns]
    df.columns = new_row

    df = df.loc[(df['conversion_process_name'] != 'DEBUG') & (df['conversion_process_name'].notnull())]
    df['cs'] = df[['conversion_process_name', 'commodity_in', 'commodity_out']].agg('@'.join, axis=1)

    populated_params = {cs: None for cs in cs_selection}
    for i in range(len(cs_selection)):
        df_filtered = df.loc[df['cs'] == cs_selection[i]]
        populated_params[cs_selection[i]] = df_filtered.iloc[:, 10:-3].dropna(axis=1).columns.to_list()

    return populated_params

def get_values(techmap_file, selection_dict):
    tmap = pd.ExcelFile(techmap_file)
    df = pd.read_excel(tmap,"ConversionSubProcess")
    units = df.iloc[1,:]
    df = df.loc[(df['conversion_process_name'] != 'DEBUG') & (df['conversion_process_name'].notnull())]
    df['cs'] = df[['conversion_process_name', 'commodity_in', 'commodity_out']].agg('@'.join, axis=1)
    result = {}
    
    for key, value in selection_dict.items():
        df_filtered = df.loc[df['cs'] == key]
        result[key] = []
        for param in value:
            result[key].append([param, df_filtered[param].values[0], units[param]])

    return result

def get_params_general_prompt_template() -> PromptTemplate:
    return PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a specialist at identifying the correct way to modify a model based on the user's USER_INPUT. \n
        
        You are part of a tool where there are two other agents ready to execute their specific tasks
        related to model modification. Your goal is to identify which one to use depending on the user's
        input. \n
        
        The two possibilities are:
        1. The user provides one or more specific instructions of modifications that should be done to the model,
        with the selection of specific values for each modification;
        2. The user asks for a specific scenario where the tool should decide which parameters and subprocesses
        should be modified, as well as deciding the correct value to change the combinations to. \n
        
        Your output must be a JSON object with a single key called 'parametrization_type', where your options are
        'defined' for the first case and 'undefined' for the second. These are your only possibilities. \n
        
        Always use double quotes in the JSON object. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT: {user_input} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["user_input"],
    )

def get_select_cs_prompt_template() -> PromptTemplate:
    return PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a specialist at analizing the USER_INPUT and determining the correct conversion
        suprocesses that should be modified in a energy system to achieve the user request. \n
        
        Your selection should come from the CONVERSION_SUBPROCESSES list provided, you are not
        allowed to guess names of conversion subprocesses or modify the entries of the list before
        returning them. You MUST use them as they are in the provided list. \n
        
        There are two possibilities of ways for you to choose the correct subprocesses:
        1. The user specified what he wants to modify in the model, in this case you should try
        to match what was requested and nothing else;
        2. The user requested a general scenario in which you will be responsible to decide
        which subprocesses should be modified. \n
        
        Your output must be a JSON object with a single key 'cs_selection' that will contain a
        single list with all entries that you selected from CONVERSION_SUBPROCESSES. \n
        
        Always use double quotes in the JSON object. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT: {user_input} \n
        CONVERSION_SUBPROCESSES: {cs_list} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["user_input","cs_list"],
    )
    
def get_select_params_prompt_template() -> PromptTemplate:
    return PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a specialist at analizing the USER_INPUT and determining the correct parameters
        that should be modified in a energy system to achieve the user request. \n
        
        Your selection should come from the PARAMETERS dictionary provided, you are not
        allowed to guess names of parameters or modify the entries of the list before
        returning them. You MUST use them as they are in the provided list. \n
        
        The PARAMETERS dictionary shows for each conversion subprocess the available parameters,
        you should select from among the ones available for that specific subprocess. The format
        in which each parameter is shown is 'param_name - description', you must output only the
        param_name, never the description. \n
        
        There are two possibilities of ways for you to choose the correct parameters:
        1. The user specified what he wants to modify in the model, in this case you should try
        to match what was requested and nothing else;
        2. The user requested a general scenario in which you will be responsible to decide
        which parameters should be modified. \n
        
        Your output must be a JSON object with a single key 'param_selection' that will contain a
        single dictionary where the keys are each of the conversion subprocesses and the values
        are the list of selected parameters for each of them. \n
        
        Always use double quotes in the JSON object. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT: {user_input} \n
        PARAMETERS: {param_list} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["user_input","param_list"],
    )

def get_new_values_prompt_template() -> PromptTemplate:
    return PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are a specialist at defining new values for model parameters based on the USER_INPUT. \n
        
        You have access to all values to be modified in CURRENT_VALUES, they are organized as a
        dictionary, where each key represents a conversion subprocess and the value of each key
        is a list of lists containing all of the current values. Each sublist of this list is
        organized as follows: [param_name, current_value, unit]. \n
        
        Your job is to understand the USER_INPUT to modify the 'current_value' in this structure.
        You will output a JSON object that contains two keys, 'success' and 'values', where 'success'
        should receive a boolean (always lower case) and 'values' should receive the same structure
        as CURRENT_VALUES but with 'current_value' updated for the new value in all entries. \n
        
        You have some options of how to update the value based on USER_INPUT:
        1. The user specified a value, in this case you need to use the exact values that were asked
        by the user;
        2. The user didn't specify a value but gave some kind of idea of the modification, for example
        asked you to double the value, to reduce by 30%, to remove (in this case the value should go to 0),
        in this case you need to figure out by how much the value will be modified;
        3. The user didn't specify a value but asked for some indirect value related to other piece of
        information, in this case you can check CONTEXT for the necessary information;
        4. In case you couldn't categorize the request as any of the past 3 cases, you should mark
        'success' as false and 'values' as an empty dict. \n
        
        There are two types of values that you can receive, numbers or lists. If the value is a number
        it's simple, the number represents the value of that parameter, however, if you receive a list
        (which will be in form of a string and should also be outputed as a string) it represents the
        variation of the parameter by year, for example '[2015 10; 2030 20; 2050 40]'. You may modify
        the available years and the values, as long as you keep the format and the output as string. \n
        
        Always use double quotes in the JSON object. \n

        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT: {user_input} \n
        CONTEXT: {context} \n
        CURRENT_VALUES: {current_values} \n
        Answer:
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """,
        input_variables=["user_input","context","current_values"],
    )

params_general_prompt = get_params_general_prompt_template()
select_cs_prompt = get_select_cs_prompt_template()
select_params_prompt = get_select_params_prompt_template()
new_values_prompt = get_new_values_prompt_template()

params_general_chain = params_general_prompt | json_model | JsonOutputParser()
select_cs_chain = select_cs_prompt | json_model |JsonOutputParser()
select_params_chain = select_params_prompt | json_model |JsonOutputParser()
new_values_chain = new_values_prompt | json_model | JsonOutputParser()

user_input = 'What happens if we reduce the residual capacity of onshore residual wind turbines?'
#user_input = 'modify the energy cost of oil import to be 10'

params, CSs = get_params_and_cs_list('../CESM/Data/Techmap/DEModel.xlsx')

llm_output = params_general_chain.invoke({"user_input": user_input})
parametrization_type = llm_output['parametrization_type']

cs_selection = select_cs_chain.invoke({"user_input": user_input, "cs_list": CSs})

if parametrization_type == 'defined':
    available_parameters = {cs: params for cs in cs_selection['cs_selection']}
else:
    available_parameters = get_populated_params_and_cs_list('../CESM/Data/Techmap/DEModel.xlsx', cs_selection['cs_selection'])

print(llm_output)
print(cs_selection)
print(available_parameters)

llm_output = select_params_chain.invoke({"user_input": user_input, "param_list": available_parameters})
param_selection = llm_output['param_selection']

print(llm_output)

current_values = get_values('../CESM/Data/Techmap/DEModel.xlsx', param_selection)

llm_output = new_values_chain.invoke({"user_input": user_input, "context": [], "current_values": current_values})

print(llm_output)

{'parametrization_type': 'undefined'}
{'cs_selection': ['PP_WindOn_Res@Dummy@Electricity', 'PP_WindOn_New@Dummy@Electricity']}
{'PP_WindOn_Res@Dummy@Electricity': ['cap_res_max - Max residual cap', 'cap_res_min - Min residual cap', 'cap_max - Max active capacity', 'opex_cost_power - Fixed OM cost'], 'PP_WindOn_New@Dummy@Electricity': ['cap_res_max - Max residual cap', 'cap_max - Max active capacity', 'technical_lifetime - Techninal Lifetime', 'opex_cost_power - Fixed OM cost', 'capex_cost_power - Investment Cost  power']}
{'param_selection': {'PP_WindOn_Res@Dummy@Electricity': ['cap_res_max'], 'PP_WindOn_New@Dummy@Electricity': []}}
{'success': True, 'values': {'PP_WindOn_Res@Dummy@Electricity': [['cap_res_max', '[2016 22.7;2030 15;2040 0]', 'GW']], 'PP_WindOn_New@Dummy@Electricity': []}}


In [26]:
def get_prompt_template() -> PromptTemplate:
    return PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an expert at understanding the USER_INPUT and modifying ACTION_HISTORY to select
        the necessary actions to be taken to fulfill the user's request. \n
        
        ACTION_HISTORY provides you with the available actions to be taken, your job is to modify the
        dictionary to select the actions to take. You'll receive all actions as 'no', to select an action
        simply change that to 'yes', the rest of the pipeline will take care of executing them. \n
        
        The actions are 'modify' to modify the model, 'run' to run the modified model, 'compare' to
        compare the original results with the ones of the modified model, 'plot' to show the visualization
        of the new results to the user, and 'consult' to check details regarding the construction of the model
        (not the details of de modeling values or results, only the theory behind the model). \n
        
        There are two types of USER_INPUT:
        1. Gives you a direct command related to one or more of the available actions, such as asking you
        to modify a value, or to run a model, to plot some specific data, compare the already available
        results or consult details about the modeling process. In this case, you should only request the
        actions the user has asked;
        2. Gives you a scenario without specifying any command, asking for details about the scenario
        for example. In this case you need to modify, run, analyze and plot the results to the user. \n
        
        IMPORTANT if the user is just request you to modify something without asking you to run the model, then
        you should assume you should only modify it. If the user actually wanted to run, he will ask later. \n
        
        You must output a JSON object with the modified dictionary. \n
        
        Always use double quotes in the JSON object. \n
        
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT : {user_input} \n
        ACTION_HISTORY: {action_history} \n
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["user_input", "action_history"],
    )
    
prompt = get_prompt_template()
llm_chain = prompt | json_model | JsonOutputParser()

user_input = 'modify the CO2 limit to be half of what it is today'
user_input = 'compare the model results'
user_input = 'what happens if we kill the co2 limit after 2030?'
user_input = 'kill the co2 limit after 2030'
user_input = 'what would change if I dont want wind turbines?'
action_history = {'modify': 'no', 'run': 'no', 'compare': 'no', 'plot': 'no', 'consult': 'no'}

llm_chain.invoke({"user_input": user_input, "action_history": action_history})

{'modify': 'yes',
 'run': 'yes',
 'compare': 'yes',
 'plot': 'yes',
 'consult': 'no'}

In [33]:
import pickle

with open("../metadata/chat_history.pkl", "rb") as f:
    history = pickle.load(f)

history

[{'role': 'user',
  'content': 'what are the dominant changes in 2050 compared to now?\n'}]

In [26]:
def get_prompt_template() -> PromptTemplate:
    return PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are responsible of verifying if the USER_INPUT is in another language other
        than english, if so, translate the input. If the text is already in english
        simply output the same text. \n
        
        Your output must be a JSON object with two keys, 'language' and 'input', where
        'language' is the source language the user wrote and 'input' is the translated
        input. \n
        
        <|eot_id|><|start_header_id|>user<|end_header_id|>
        USER_INPUT : {user_input} \n
        <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["user_input"],
    )
    
prompt = get_prompt_template()
llm_chain = prompt | test_json_model | JsonOutputParser()

user_input = 'O que acontece se a gente reduzir o custo de turbinas eólicas offshore?'
user_input = 'cosa succede se riduciamo il costo delle turbine eoliche offshore?'
user_input = 'Was passiert, wenn wir die Kosten für Offshore-Windturbinen senken?'
user_input = 'What happens if we lower the cost of offshore wind turbines?'
user_input = 'que se passe-t-il si nous réduisons le coût des éoliennes en mer ?'
user_input = 'mi történik, ha csökkentjük a tengeri szélturbinák költségeit?'
user_input = 'apa yang terjadi jika kita mengurangi biaya turbin angin lepas pantai?'
user_input = 'τι θα συμβεί αν μειώσουμε το κόστος των υπεράκτιων ανεμογεννητριών;'
user_input = 'Mitä tapahtuu, jos merituulivoimaloiden kustannuksia alennetaan?'
user_input = 'ماذا يحدث إذا خفضنا تكلفة توربينات الرياح البحرية؟'
user_input = '如果我们降低海上风力涡轮机的成本会怎样？'
user_input = '洋上風力タービンのコストを下げたらどうなるか？'
user_input = "Kio estas via nomo? Kiel vi fartas?"

llm_chain.invoke({"user_input": user_input})

{'language': 'esperanto', 'input': 'What is your name? How are you?'}